In [1]:
# Install SAM
#!pip install segment-anything

#!pip install git+https://github.com/ChaoningZhang/MobileSAM.git
# Download the weights (only 40MB)
#!wget https://raw.githubusercontent.com/ChaoningZhang/MobileSAM/master/weights/mobile_sam.pt

In [ ]:
"""
CAMDF-Net: Context-Aware Multi-Domain Fusion Network
"""
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import torch.optim as optim
import cv2
import numpy as np
from PIL import Image
import yaml
import json
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Any
from dataclasses import dataclass, field, asdict
from tqdm import tqdm
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
from sklearn.metrics import (
    accuracy_score, precision_recall_fscore_support,
    confusion_matrix, roc_auc_score, roc_curve,
    precision_recall_curve, average_precision_score
)
from scipy.ndimage import uniform_filter
from scipy import stats as scipy_stats
import datetime
import os
import random
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# CONFIGURATION SYSTEM
# ============================================================================
@dataclass
class DatasetConfig:
    """Dataset configuration with defaults"""
    data_root: str = "/kaggle/input/plantvillage-dataset/color"
    train_ratio: float = 0.8
    val_ratio: float = 0.1
    test_ratio: float = 0.1
    batch_size: int = 16  # Increased from 1 to avoid batch norm issues
    num_workers: int = 4
    pin_memory: bool = True
    mask_method: str = "ensemble"
    augment: bool = True
    image_size: int = 512
    use_advanced_augmentation: bool = True
    augment_prob: float = 0.5
    color_jitter_brightness: float = 0.3
    color_jitter_contrast: float = 0.3
    color_jitter_saturation: float = 0.3
    color_jitter_hue: float = 0.15
    rotation_degrees: int = 30
    use_tta: bool = True
    tta_transforms: int = 5
    preserve_original_size_eval: bool = True

@dataclass
class ModelConfig:
    """Model architecture configuration"""
    name: str = "camdfnet_base"
    num_classes: int = None  # Will be set automatically from dataset
    hidden_dim: int = 768
    use_fpn: bool = True
    use_se: bool = True
    use_sarm: bool = True
    dropout_rate: float = 0.1
    pretrained: bool = True
    return_attention_maps: bool = True
    save_all_activations: bool = True

@dataclass
class TrainingConfig:
    """Training hyperparameters"""
    epochs: int = 200
    learning_rate: float = 1e-4
    weight_decay: float = 1e-5
    optimizer: str = "adamw"
    scheduler: str = "cosine"
    warmup_epochs: int = 10
    gradient_clip: float = 1.0
    early_stopping_patience: int = 50
    mixed_precision: bool = True
    use_mixup: bool = True
    mixup_alpha: float = 0.2
    use_cutmix: bool = True
    cutmix_alpha: float = 1.0
    augmentation_prob: float = 0.5
    split: str = 'test'

@dataclass
class LossConfig:
    """Loss function configuration"""
    lambda_dice: float = 1.0
    lambda_bce: float = 1.0
    lambda_cls: float = 1.0
    lambda_mask_reg: float = 0.5
    lambda_aux: float = 0.5
    use_focal: bool = True
    focal_alpha: float = 0.25
    focal_gamma: float = 2.0
    use_class_weights: bool = True
    label_smoothing: float = 0.2

@dataclass
class SystemConfig:
    """System-level configuration"""
    device: str = "cuda"
    seed: int = 42
    checkpoint_dir: str = "./runnings"
    log_dir: str = "./logs"
    output_dir: str = "./results"
    save_frequency: int = 25
    verbose: bool = True
    experiment_name: str = "camdfnet"
    save_all_intermediates: bool = True

@dataclass
class CAMDFNetConfig:
    """Complete system configuration"""
    dataset: DatasetConfig = field(default_factory=DatasetConfig)
    model: ModelConfig = field(default_factory=ModelConfig)
    training: TrainingConfig = field(default_factory=TrainingConfig)
    loss: LossConfig = field(default_factory=LossConfig)
    system: SystemConfig = field(default_factory=SystemConfig)
    
    @classmethod
    def from_yaml(cls, path: str) -> 'CAMDFNetConfig':
        with open(path, 'r') as f:
            data = yaml.safe_load(f)
        return cls(
            dataset=DatasetConfig(**data.get('dataset', {})),
            model=ModelConfig(**data.get('model', {})),
            training=TrainingConfig(**data.get('training', {})),
            loss=LossConfig(**data.get('loss', {})),
            system=SystemConfig(**data.get('system', {}))
        )
    
    def to_yaml(self, path: str):
        data = {
            'dataset': asdict(self.dataset),
            'model': asdict(self.model),
            'training': asdict(self.training),
            'loss': asdict(self.loss),
            'system': asdict(self.system)
        }
        with open(path, 'w') as f:
            yaml.dump(data, f, default_flow_style=False)

# ============================================================================
# SPP
# ============================================================================
class SpectralPreProcessor(nn.Module):
    def __init__(self, in_channels: int = 3, freq_ratios: List[float] = [0.2, 0.5, 1.0]):
        super().__init__()
        self.ratios = freq_ratios
        self.filters = nn.ParameterList([
            nn.Parameter(torch.randn(1, in_channels, 1, 1, dtype=torch.complex64))
            for _ in freq_ratios
        ])
        for f in self.filters:
            nn.init.normal_(f.real, mean=1.0, std=0.01)
            nn.init.normal_(f.imag, mean=0.0, std=0.01)
        self.activations = {}
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        b, c, h, w = x.shape
        x_fft = torch.fft.rfft2(x, norm='ortho')
        out_spatial = torch.zeros_like(x)
        for i, ratio in enumerate(self.ratios):
            h_lim = max(1, int(x_fft.shape[2] * ratio))
            w_lim = max(1, int(x_fft.shape[3] * ratio))
            mask = torch.zeros_like(x_fft)
            mask[:, :, :h_lim, :w_lim] = x_fft[:, :, :h_lim, :w_lim]
            out_spatial += torch.fft.irfft2(mask * self.filters[i], s=(h, w), norm='ortho')
        
        self.activations['spectral_output'] = out_spatial.detach()
        return out_spatial / len(self.ratios)

class PyramidPoolingModule(nn.Module):
    def __init__(self, in_channels: int, pool_sizes: List[int] = [1, 2, 3, 6]):
        super().__init__()
        out_channels = in_channels // len(pool_sizes)
        self.stages = nn.ModuleList([
            nn.Sequential(
                nn.AdaptiveAvgPool2d(size),
                nn.Conv2d(in_channels, out_channels, 1, bias=False),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(inplace=True)
            ) for size in pool_sizes
        ])
        self.activations = {}
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        h, w = x.shape[2:]
        pyramids = [x]
        for i, stage in enumerate(self.stages):
            pooled = stage(x)
            pyramids.append(F.interpolate(pooled, size=(h, w), mode='bilinear', align_corners=True))
            self.activations[f'pool_stage_{i}'] = pooled.detach()
        
        output = torch.cat(pyramids, dim=1)
        self.activations['pyramid_output'] = output.detach()
        return output

class AttentionRefinementModule(nn.Module):
    """Refine segmentation masks using attention"""
    def __init__(self, in_channels: int):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels, in_channels // 2, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(in_channels // 2)
        self.conv2 = nn.Conv2d(in_channels // 2, 1, 1)
        self.activations = {}
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x1 = F.relu(self.bn1(self.conv1(x)))
        attention = torch.sigmoid(self.conv2(x1))
        
        self.activations['attention_input'] = x.detach()
        self.activations['attention_output'] = attention.detach()
        
        return attention

class CAMDFNet(nn.Module):
    def __init__(self, config: ModelConfig):
        super().__init__()
        self.config = config
        self.activations = {}
        self.gradients = {}
        is_large = "large" in config.name.lower()
        
        if is_large:
            self.vit = models.vit_l_16(weights=models.ViT_L_16_Weights.IMAGENET1K_V1)
            self.hidden_dim, self.num_heads = 1024, 16
            eff = models.efficientnet_b7(weights=models.EfficientNet_B7_Weights.IMAGENET1K_V1)
            cnn_ch = 224
        else:
            self.vit = models.vit_b_16(weights=models.ViT_B_16_Weights.IMAGENET1K_V1)
            self.hidden_dim, self.num_heads = 768, 12
            eff = models.efficientnet_b4(weights=models.EfficientNet_B4_Weights.IMAGENET1K_V1)
            cnn_ch = 160
        
        self.vit_layers = self.vit.encoder.layers
        self.spectral_pre = SpectralPreProcessor()
        self.cnn_backbone = nn.Sequential(*list(eff.features.children())[:6])
        self.proj2 = nn.Conv2d(cnn_ch, self.hidden_dim, 1)
        self.pyramid_pool = PyramidPoolingModule(self.hidden_dim)
        
        self.attention_refine = AttentionRefinementModule(self.hidden_dim * 2)
        
        # Segmentation head
        self.segmentation_head = nn.Sequential(
            nn.Conv2d(self.hidden_dim * 2, 256, 3, padding=1), nn.ReLU(inplace=True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1), nn.BatchNorm2d(128), nn.ReLU(inplace=True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1), nn.BatchNorm2d(64), nn.ReLU(inplace=True),
            nn.ConvTranspose2d(64, 32, 4, 2, 1), nn.BatchNorm2d(32), nn.ReLU(inplace=True),
            nn.ConvTranspose2d(32, 16, 4, 2, 1), nn.BatchNorm2d(16), nn.ReLU(inplace=True),
            nn.Conv2d(16, 1, 1)
        )
        
        self.cross_attention = nn.MultiheadAttention(self.hidden_dim, num_heads=self.num_heads, batch_first=False)
        
        # Classifier - IMPORTANT: num_classes is set automatically from dataset
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d(1), nn.Flatten(),
            nn.Linear(self.hidden_dim, config.num_classes)
        )
        
        self.uncertainty_head = nn.Sequential(
            nn.AdaptiveAvgPool2d(1), nn.Flatten(),
            nn.Linear(self.hidden_dim, 1), nn.Sigmoid()
        )
        
        # Register hooks for all intermediate layers
        self._register_all_hooks()
    
    def _register_all_hooks(self):
        def save_activation(name):
            def hook(module, input, output):
                if isinstance(output, tuple):
                    if len(output) == 2:
                        if output[0] is not None and hasattr(output[0], 'detach'):
                            self.activations[name + '_output'] = output[0].detach()
                        if output[1] is not None and hasattr(output[1], 'detach'):
                            self.activations[name + '_weights'] = output[1].detach()
                    else:
                        for i, out in enumerate(output):
                            if out is not None and hasattr(out, 'detach'):
                                self.activations[f'{name}_tuple_{i}'] = out.detach()
                else:
                    if output is not None and hasattr(output, 'detach'):
                        self.activations[name] = output.detach()
            return hook
        
        def save_gradient(name):
            def hook(module, grad_input, grad_output):
                if isinstance(grad_output, tuple):
                    if len(grad_output) > 0 and grad_output[0] is not None:
                        self.gradients[name] = grad_output[0].detach()
                elif grad_output is not None and hasattr(grad_output, 'detach'):
                    self.gradients[name] = grad_output.detach()
            return hook
        
        self.spectral_pre.register_forward_hook(save_activation('spectral_pre'))
        
        for i, layer in enumerate(self.cnn_backbone):
            layer.register_forward_hook(save_activation(f'cnn_layer_{i}'))
        
        self.proj2.register_forward_hook(save_activation('proj2'))
        self.pyramid_pool.register_forward_hook(save_activation('pyramid_pool'))
        self.attention_refine.register_forward_hook(save_activation('attention_refine'))
        
        for i, layer in enumerate(self.segmentation_head):
            layer.register_forward_hook(save_activation(f'seg_layer_{i}'))
        
        self.cross_attention.register_forward_hook(save_activation('cross_attention'))
        self.classifier.register_forward_hook(save_activation('classifier'))
        self.uncertainty_head.register_forward_hook(save_activation('uncertainty_head'))
    
    def forward(self, x: torch.Tensor):
        b, _, h, w = x.shape
        
        self.activations.clear()
        self.activations['input'] = x.detach()
        
        x_res = F.interpolate(x, size=(224, 224), mode='bilinear', align_corners=True)
        x_spec = self.spectral_pre(x_res)
        
        v = self.vit
        x_v = v._process_input(x_spec)
        n = x_v.shape[0]
        cls_token = v.class_token.expand(n, -1, -1)
        x_v = torch.cat((cls_token, x_v), dim=1)
        x_v = v.encoder(x_v)
        
        vit_spatial = x_v[:, 1:].transpose(1, 2).reshape(b, self.hidden_dim, 14, 14)
        self.activations['vit_spatial'] = vit_spatial.detach()
        self.activations['vit_sequence'] = x_v.detach()
        
        # Get pyramid pooled features
        pooled_features = self.pyramid_pool(vit_spatial)
        
        # Get attention weights for mask refinement
        attention_weights = self.attention_refine(pooled_features)
        
        # Generate mask with attention refinement
        mask_logits = self.segmentation_head(pooled_features)
        attention_weights_upsampled = F.interpolate(attention_weights, size=(224, 224), mode='bilinear', align_corners=True)
        refined_mask = mask_logits * attention_weights_upsampled
        self.activations['mask_logits'] = mask_logits.detach()
        self.activations['refined_mask'] = refined_mask.detach()
        
        gated_x = x_spec * (0.5 + 0.5 * torch.sigmoid(refined_mask))
        self.activations['gated_x'] = gated_x.detach()
        
        cnn_features = self.cnn_backbone(gated_x)
        self.activations['cnn_features_raw'] = cnn_features.detach()
        
        cnn_aligned = F.interpolate(self.proj2(cnn_features), size=(14, 14), mode='bilinear', align_corners=True)
        self.activations['cnn_aligned'] = cnn_aligned.detach()
        
        cnn_seq = cnn_aligned.flatten(2).permute(2, 0, 1)
        vit_seq = x_v.permute(1, 0, 2)
        
        fused_seq, cross_weights = self.cross_attention(cnn_seq, vit_seq, vit_seq, need_weights=True)
        
        fused_feat = fused_seq.permute(1, 2, 0).reshape(b, self.hidden_dim, 14, 14)
        self.activations['fused_features'] = fused_feat.detach()
        
        cls_logits = self.classifier(fused_feat)
        uncertainty = self.uncertainty_head(fused_feat)
        mask_upscaled = F.interpolate(refined_mask, size=(h, w), mode='bilinear', align_corners=True)
        
        self.activations['cls_logits'] = cls_logits.detach()
        self.activations['uncertainty'] = uncertainty.detach()
        self.activations['final_mask'] = mask_upscaled.detach()
        
        return {
            'logits': cls_logits,
            'mask': mask_upscaled,
            'uncertainty': uncertainty,
            'cross_attention_weights': cross_weights,
            'spectral_image': x_spec,
            'vit_features': vit_spatial,
            'cnn_features': cnn_aligned,
            'fused_features': fused_feat,
            'vit_sequence': x_v,
            'attention_weights': attention_weights
        }

    @torch.no_grad()
    def get_vit_self_attention(self, x: torch.Tensor, 
                               layer_indices: List[int] = None,
                               return_all_heads: bool = False) -> Dict[str, Any]:
        self.eval()
        b, _, h, w = x.shape
        
        x_res = F.interpolate(x, size=(224, 224), mode='bilinear', align_corners=True)
        x_spec = self.spectral_pre(x_res)
        v = self.vit
        
        x_v = v._process_input(x_spec)
        n = x_v.shape[0]
        cls_token = v.class_token.expand(n, -1, -1)
        x_v = torch.cat((cls_token, x_v), dim=1)
        x_v = x_v + v.encoder.pos_embedding
        
        if layer_indices is None:
            layer_indices = list(range(len(v.encoder.layers)))
        
        attention_maps = {}
        
        current_x = x_v
        for idx, layer in enumerate(v.encoder.layers):
            if idx in layer_indices:
                norm_x = layer.ln_1(current_x)
                qkv = F.linear(norm_x, layer.self_attention.in_proj_weight, 
                              layer.self_attention.in_proj_bias)
                
                head_dim = self.hidden_dim // self.num_heads
                qkv = qkv.reshape(b, -1, 3, self.num_heads, head_dim).permute(2, 0, 3, 1, 4)
                q, k, v_ = qkv[0], qkv[1], qkv[2]
                
                attn = (q @ k.transpose(-2, -1)) * (head_dim ** -0.5)
                attn = attn.softmax(dim=-1)
                
                if return_all_heads:
                    head_attentions = {}
                    for head_idx in range(self.num_heads):
                        cls_attn_head = attn[:, head_idx, 0, 1:]
                        cls_attn_map = cls_attn_head.reshape(b, 14, 14)
                        
                        map_min = cls_attn_map.amin(dim=(1, 2), keepdim=True)
                        map_max = cls_attn_map.amax(dim=(1, 2), keepdim=True)
                        cls_attn_map = (cls_attn_map - map_min) / (map_max - map_min + 1e-8)
                        
                        head_attentions[f'head_{head_idx}'] = cls_attn_map
                    
                    attention_maps[f'layer_{idx}'] = head_attentions
                else:
                    attn_avg = attn.mean(dim=1)
                    cls_attn = attn_avg[:, 0, 1:]
                    cls_attn_map = cls_attn.reshape(b, 14, 14)
                    
                    map_min = cls_attn_map.amin(dim=(1, 2), keepdim=True)
                    map_max = cls_attn_map.amax(dim=(1, 2), keepdim=True)
                    cls_attn_map = (cls_attn_map - map_min) / (map_max - map_min + 1e-8)
                    
                    attention_maps[f'layer_{idx}'] = cls_attn_map
            
            current_x = layer(current_x)
        
        return attention_maps
    
    def get_all_activations(self) -> Dict[str, torch.Tensor]:
        return self.activations.copy()
    
    def get_all_gradients(self) -> Dict[str, torch.Tensor]:
        return self.gradients.copy()

# ============================================================================
# MASK GENERATOR
# ============================================================================
class MaskGenerator:
    @staticmethod
    def generate_hsv_disease_mask(image: np.ndarray) -> np.ndarray:
        hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
        
        lower_brown = np.array([10, 30, 20])
        upper_brown = np.array([30, 180, 150])
        brown_mask = cv2.inRange(hsv, lower_brown, upper_brown)
        
        lower_yellow = np.array([20, 40, 100])
        upper_yellow = np.array([35, 255, 255])
        yellow_mask = cv2.inRange(hsv, lower_yellow, upper_yellow)
        
        lower_dark = np.array([0, 0, 0])
        upper_dark = np.array([180, 255, 80])
        dark_mask = cv2.inRange(hsv, lower_dark, upper_dark)
        
        disease_mask = cv2.bitwise_or(brown_mask, yellow_mask)
        disease_mask = cv2.bitwise_or(disease_mask, dark_mask)
        
        return disease_mask
    
    @staticmethod
    def generate_lab_disease_mask(image: np.ndarray) -> np.ndarray:
        lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
        l_channel, a_channel, b_channel = cv2.split(lab)
        
        _, a_thresh = cv2.threshold(a_channel, 135, 255, cv2.THRESH_BINARY)
        _, b_thresh = cv2.threshold(b_channel, 135, 255, cv2.THRESH_BINARY)
        
        lab_mask = cv2.bitwise_or(a_thresh, b_thresh)
        
        return lab_mask
    
    @staticmethod
    def generate_edge_disease_mask(image: np.ndarray) -> np.ndarray:
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        
        bilateral = cv2.bilateralFilter(gray, 9, 75, 75)
        
        edges = cv2.Canny(bilateral, 30, 100)
        
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
        dilated = cv2.dilate(edges, kernel, iterations=2)
        
        contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        edge_mask = np.zeros_like(gray)
        cv2.drawContours(edge_mask, contours, -1, 255, thickness=cv2.FILLED)
        
        return edge_mask
    
    @staticmethod
    def generate_texture_disease_mask(image: np.ndarray) -> np.ndarray:
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        
        gabor_responses = []
        for theta in [0, 45, 90, 135]:
            kernel = cv2.getGaborKernel((21, 21), 5, np.deg2rad(theta), 10, 0.5, 0, ktype=cv2.CV_32F)
            filtered = cv2.filter2D(gray, cv2.CV_8UC3, kernel)
            gabor_responses.append(filtered)
        
        gabor_combined = np.max(np.stack(gabor_responses), axis=0).astype(np.uint8)
        
        _, texture_mask = cv2.threshold(gabor_combined, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        
        return texture_mask
    
    @staticmethod
    def generate_statistical_mask(image: np.ndarray) -> np.ndarray:
        lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
        
        masks = []
        for channel in cv2.split(lab):
            local_mean = uniform_filter(channel.astype(float), size=15)
            local_var = uniform_filter(channel.astype(float)**2, size=15) - local_mean**2
            local_var = np.maximum(local_var, 0)
            
            var_range = local_var.max() - local_var.min()
            if var_range < 1e-8:
                var_norm = np.zeros_like(local_var, dtype=np.uint8)
            else:
                var_norm = ((local_var - local_var.min()) / (var_range + 1e-8) * 255).astype(np.uint8)
            _, var_mask = cv2.threshold(var_norm, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            masks.append(var_mask)
        
        stat_mask = cv2.bitwise_or(masks[0], masks[1])
        stat_mask = cv2.bitwise_or(stat_mask, masks[2])
        
        return stat_mask
    
    @staticmethod
    def ensemble_mask_generation(image: np.ndarray, methods: Optional[List[str]] = None) -> np.ndarray:
        if methods is None:
            methods = ['hsv', 'lab', 'edge', 'texture', 'statistical']
        
        masks = []
        
        if 'hsv' in methods:
            masks.append(MaskGenerator.generate_hsv_disease_mask(image))
        if 'lab' in methods:
            masks.append(MaskGenerator.generate_lab_disease_mask(image))
        if 'edge' in methods:
            masks.append(MaskGenerator.generate_edge_disease_mask(image))
        if 'texture' in methods:
            masks.append(MaskGenerator.generate_texture_disease_mask(image))
        if 'statistical' in methods:
            masks.append(MaskGenerator.generate_statistical_mask(image))
        
        mask_stack = np.stack([m / 255.0 for m in masks], axis=0)
        vote_mask = (mask_stack.mean(axis=0) > 0.5).astype(np.uint8) * 255
        
        vote_mask = MaskGenerator.post_process_mask(vote_mask, image)
        
        return vote_mask
    
    @staticmethod
    def post_process_mask(mask: np.ndarray, original_image: np.ndarray) -> np.ndarray:
        kernel_small = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
        kernel_large = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
        
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel_small, iterations=2)
        
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel_large, iterations=2)
        
        num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(mask, connectivity=8)
        min_size = int((mask.shape[0] * mask.shape[1]) * 0.001)
        
        filtered_mask = np.zeros_like(mask)
        for i in range(1, num_labels):
            if stats[i, cv2.CC_STAT_AREA] >= min_size:
                filtered_mask[labels == i] = 255
        
        filtered_mask = cv2.GaussianBlur(filtered_mask, (5, 5), 0)
        _, filtered_mask = cv2.threshold(filtered_mask, 127, 255, cv2.THRESH_BINARY)
        
        return filtered_mask

# ============================================================================
# CUSTOM COLLATE FUNCTION
# ============================================================================
def custom_collate_fn(batch):
    images = torch.stack([item[0] for item in batch])
    
    mask_shapes = [item[1].shape for item in batch]
    all_same_shape = all(s == mask_shapes[0] for s in mask_shapes)
    
    if all_same_shape:
        masks = torch.stack([item[1] for item in batch])
    else:
        masks = [item[1] for item in batch]
    
    labels = torch.tensor([item[2] for item in batch], dtype=torch.long)
    
    original_sizes = [item[3] for item in batch]
    
    return images, masks, labels, original_sizes

# ============================================================================
# DATASET WITH IMPROVED MASK GENERATION
# ============================================================================
class CottonDiseaseDataset(Dataset):
    def __init__(self, config, split: str = 'train'):
        self.config = config
        self.split = split
        self.data_root = Path(config.data_root)
        self.samples = []
        self.classes = sorted([d.name for d in self.data_root.iterdir() if d.is_dir()])
        self.class_to_idx = {cls: idx for idx, cls in enumerate(self.classes)}
        self.mask_generator = MaskGenerator()
        print(f"Classes found: {len(self.classes)}")
        print(f"Classes: {self.classes}")
        
        for class_name in self.classes:
            class_dir = self.data_root / class_name
            for img_path in sorted(list(class_dir.glob('*.*')) + list(class_dir.glob('*.png'))):
                self.samples.append((str(img_path), self.class_to_idx[class_name]))
        
        print(f"Total samples: {len(self.samples)}")

    def __getitem__(self, idx: int) -> Tuple[torch.Tensor, torch.Tensor, int, Tuple[int, int]]:
        img_path, label = self.samples[idx]
        
        image = Image.open(img_path).convert('RGB')
        original_size = image.size
        
        image_np = np.array(image)
        mask = self.generate_mask(image_np)
        
        mask_pil = Image.fromarray((mask * 255).astype(np.uint8)).convert('L')
        
        if self.split == 'train' and getattr(self.config, 'augment', False):
            image_transformed, mask_transformed = self.apply_augmentations(image, mask_pil)
        else:
            image_transformed = transforms.Resize((self.config.image_size, self.config.image_size))(image)
            if self.split in ['test', 'val'] and getattr(self.config, 'preserve_original_size_eval', True):
                mask_transformed = mask_pil
            else:
                mask_transformed = transforms.Resize(
                    (self.config.image_size, self.config.image_size), 
                    interpolation=transforms.InterpolationMode.NEAREST
                )(mask_pil)
        
        image_tensor = transforms.ToTensor()(image_transformed)
        mask_tensor = transforms.ToTensor()(mask_transformed)
        
        normalize = transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        image_tensor = normalize(image_tensor)
        
        return image_tensor, mask_tensor, label, original_size

    def apply_augmentations(self, image: Image.Image, mask: Image.Image) -> Tuple[Image.Image, Image.Image]:
        if random.random() < 0.5:
            image = transforms.functional.hflip(image)
            mask = transforms.functional.hflip(mask)
        
        if random.random() < 0.5:
            image = transforms.functional.vflip(image)
            mask = transforms.functional.vflip(mask)
        
        if random.random() < 0.5:
            angle = random.uniform(
                -getattr(self.config, 'rotation_degrees', 30), 
                getattr(self.config, 'rotation_degrees', 30)
            )
            image = transforms.functional.rotate(image, angle, interpolation=transforms.InterpolationMode.BILINEAR)
            mask = transforms.functional.rotate(mask, angle, interpolation=transforms.InterpolationMode.NEAREST)
        
        image = transforms.Resize((self.config.image_size, self.config.image_size))(image)
        mask = transforms.Resize(
            (self.config.image_size, self.config.image_size), 
            interpolation=transforms.InterpolationMode.NEAREST
        )(mask)
        
        if random.random() < getattr(self.config, 'augment_prob', 0.5) and getattr(self.config, 'use_advanced_augmentation', False):
            color_jitter = transforms.ColorJitter(
                brightness=getattr(self.config, 'color_jitter_brightness', 0.2),
                contrast=getattr(self.config, 'color_jitter_contrast', 0.2),
                saturation=getattr(self.config, 'color_jitter_saturation', 0.2),
                hue=getattr(self.config, 'color_jitter_hue', 0.1)
            )
            image = color_jitter(image)
        
        return image, mask

    def generate_mask(self, image: np.ndarray) -> np.ndarray:
        method = getattr(self.config, 'mask_method', 'ensemble')
        
        if method == 'ensemble':
            mask = self.mask_generator.ensemble_mask_generation(image)
        elif method == 'hsv':
            mask = self.mask_generator.generate_hsv_disease_mask(image)
        elif method == 'lab':
            mask = self.mask_generator.generate_lab_disease_mask(image)
        elif method == 'edge':
            mask = self.mask_generator.generate_edge_disease_mask(image)
        elif method == 'texture':
            mask = self.mask_generator.generate_texture_disease_mask(image)
        elif method == 'statistical':
            mask = self.mask_generator.generate_statistical_mask(image)
        else:
            mask = self.mask_generator.ensemble_mask_generation(image)
        
        if method != 'ensemble':
            mask = self.mask_generator.post_process_mask(mask, image)
            
        return mask.astype(np.float32) / 255.0

    def __len__(self) -> int:
        return len(self.samples)

def create_dataloaders(config: CAMDFNetConfig) -> Tuple[DataLoader, DataLoader, DataLoader]:
    from torch.utils.data import random_split
    full_dataset = CottonDiseaseDataset(config.dataset, split=config.training.split)
    
    # Set num_classes automatically from dataset
    config.model.num_classes = len(full_dataset.classes)
    print(f"\n📊 Dataset Information:")
    print(f"  Total samples: {len(full_dataset)}")
    print(f"  Number of classes: {config.model.num_classes}")
    print(f"  Classes: {full_dataset.classes}")
    
    total_size = len(full_dataset)
    train_size = int(total_size * config.dataset.train_ratio)
    val_size = int(total_size * config.dataset.val_ratio)
    test_size = total_size - train_size - val_size
    
    generator = torch.Generator().manual_seed(config.system.seed)
    train_dataset, val_dataset, test_dataset = random_split(
        full_dataset, [train_size, val_size, test_size], generator=generator
    )
    
    print(f"  Training samples: {len(train_dataset)}")
    print(f"  Validation samples: {len(val_dataset)}")
    print(f"  Test samples: {len(test_dataset)}")
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=config.dataset.batch_size,
        shuffle=True,
        num_workers=config.dataset.num_workers,
        pin_memory=config.dataset.pin_memory,
        collate_fn=custom_collate_fn
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=config.dataset.batch_size,
        shuffle=False,
        num_workers=config.dataset.num_workers,
        pin_memory=config.dataset.pin_memory,
        collate_fn=custom_collate_fn
    )
    test_loader = DataLoader(
        test_dataset,
        batch_size=config.dataset.batch_size,
        shuffle=False,
        num_workers=config.dataset.num_workers,
        pin_memory=config.dataset.pin_memory,
        collate_fn=custom_collate_fn
    )
    
    return train_loader, val_loader, test_loader

# ============================================================================
# AUGMENTATION & LOSS FUNCTIONS
# ============================================================================
def mixup_data(x: torch.Tensor, y: torch.Tensor, alpha: float = 0.2):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    batch_size = x.size(0)
    index = torch.randperm(batch_size).to(x.device)
    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

def cutmix_data(x: torch.Tensor, y: torch.Tensor, alpha: float = 1.0):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1
    batch_size = x.size(0)
    index = torch.randperm(batch_size).to(x.device)
    _, _, H, W = x.shape
    cut_rat = np.sqrt(1. - lam)
    cut_w = int(W * cut_rat)
    cut_h = int(H * cut_rat)
    cx = np.random.randint(W)
    cy = np.random.randint(H)
    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)
    mixed_x = x.clone()
    mixed_x[:, :, bby1:bby2, bbx1:bbx2] = x[index, :, bby1:bby2, bbx1:bbx2]
    lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (W * H))
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

class DiceLoss(nn.Module):
    def forward(self, pred: torch.Tensor, target: torch.Tensor, smooth: float = 1.0) -> torch.Tensor:
        pred = torch.sigmoid(pred)
        pred_flat = pred.view(pred.size(0), -1)
        target_flat = target.view(target.size(0), -1)
        intersection = (pred_flat * target_flat).sum(dim=1)
        union = pred_flat.sum(dim=1) + target_flat.sum(dim=1)
        dice = (2.0 * intersection + smooth) / (union + smooth)
        return 1.0 - dice.mean()

class FocalLoss(nn.Module):
    def __init__(self, alpha: float = 0.25, gamma: float = 2.0):
        super().__init__()
        self.alpha = alpha
        self.gamma = gamma
    
    def forward(self, pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        ce_loss = F.cross_entropy(pred, target, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss
        return focal_loss.mean()

def _prepare_masks_for_loss(mask_pred: torch.Tensor, mask_true, device: torch.device) -> Tuple[torch.Tensor, torch.Tensor]:
    if isinstance(mask_true, torch.Tensor):
        if mask_true.shape[-2:] != mask_pred.shape[-2:]:
            mask_true = F.interpolate(mask_true.float(), size=mask_pred.shape[-2:], mode='nearest')
        return mask_pred, mask_true.to(device)
    elif isinstance(mask_true, list):
        resized_masks = []
        for m in mask_true:
            if isinstance(m, np.ndarray):
                m = torch.from_numpy(m).float()
            if m.dim() == 2:
                m = m.unsqueeze(0).unsqueeze(0)
            elif m.dim() == 3:
                m = m.unsqueeze(0)
            m_resized = F.interpolate(m.float(), size=mask_pred.shape[-2:], mode='nearest')
            resized_masks.append(m_resized.squeeze(0))
        mask_true_stacked = torch.stack(resized_masks).to(device)
        return mask_pred, mask_true_stacked
    else:
        raise TypeError(f"Unexpected mask_true type: {type(mask_true)}")

class CAMDFNetLoss(nn.Module):
    def __init__(self, config: LossConfig, class_weights: Optional[torch.Tensor] = None):
        super().__init__()
        self.config = config
        self.dice_loss = DiceLoss()
        self.focal_loss = FocalLoss(config.focal_alpha, config.focal_gamma) if config.use_focal else None
        self.class_weights = class_weights
    
    def forward(self, outputs: Dict[str, torch.Tensor], targets: Tuple) -> Dict[str, torch.Tensor]:
        images, mask_true, labels = targets[:3]
        logits = outputs['logits']
        mask_pred = outputs['mask']
        uncertainty = outputs['uncertainty']
        
        device = logits.device
        
        mask_pred_for_loss, mask_true_for_loss = _prepare_masks_for_loss(mask_pred, mask_true, device)
        
        if self.config.use_focal and self.focal_loss:
            cls_loss = self.focal_loss(logits, labels)
        else:
            cls_loss = F.cross_entropy(
                logits, labels,
                weight=self.class_weights,
                label_smoothing=self.config.label_smoothing
            )
        
        dice_loss = self.dice_loss(mask_pred_for_loss, mask_true_for_loss)
        bce_loss = F.binary_cross_entropy_with_logits(mask_pred_for_loss, mask_true_for_loss)
        mask_reg = (torch.sigmoid(mask_pred_for_loss).mean() - 0.5).abs()
        aux_loss = torch.tensor(0.0, device=device)
        
        total_loss = (
            self.config.lambda_cls * cls_loss +
            self.config.lambda_dice * dice_loss +
            self.config.lambda_bce * bce_loss +
            self.config.lambda_mask_reg * mask_reg +
            self.config.lambda_aux * aux_loss
        )
        
        return {
            'total': total_loss,
            'classification': cls_loss,
            'dice': dice_loss,
            'bce': bce_loss,
            'mask_reg': mask_reg,
            'auxiliary': aux_loss
        }

# ============================================================================
# TRAINING ENGINE WITH BATCH SIZE HANDLING
# ============================================================================
class CAMDFNetTrainer:
    def __init__(self, config: CAMDFNetConfig):
        self.config = config
        self.device = torch.device(config.system.device if torch.cuda.is_available() else 'cpu')
        Path(config.system.checkpoint_dir).mkdir(parents=True, exist_ok=True)
        Path(config.system.log_dir).mkdir(parents=True, exist_ok=True)
        
        print(f"\n🤖 Initializing CAMDF-Net Trainer")
        print(f"   Device: {self.device}")
        
        # Create dataloaders first to get number of classes
        self.train_loader, self.val_loader, self.test_loader = create_dataloaders(config)
        
        print(f"\n📊 Creating model with {config.model.num_classes} classes")
        self.model = CAMDFNet(config.model).to(self.device)
        
        # Calculate class weights
        class_weights = self._calculate_class_weights() if config.loss.use_class_weights else None
        if class_weights is not None:
            print(f"   Class weights calculated: {class_weights.cpu().numpy()}")
        
        self.criterion = CAMDFNetLoss(config.loss, class_weights)
        
        if config.training.optimizer == 'adamw':
            self.optimizer = optim.AdamW(
                self.model.parameters(),
                lr=config.training.learning_rate,
                weight_decay=config.training.weight_decay
            )
        elif config.training.optimizer == 'adam':
            self.optimizer = optim.Adam(
                self.model.parameters(),
                lr=config.training.learning_rate,
                weight_decay=config.training.weight_decay
            )
        else:
            self.optimizer = optim.SGD(
                self.model.parameters(),
                lr=config.training.learning_rate,
                momentum=0.9,
                weight_decay=config.training.weight_decay
            )
        
        if config.training.scheduler == 'cosine':
            self.scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(
                self.optimizer, T_0=10, T_mult=2
            )
        elif config.training.scheduler == 'step':
            self.scheduler = optim.lr_scheduler.StepLR(self.optimizer, step_size=30, gamma=0.1)
        else:
            self.scheduler = None
        
        self.current_epoch = 0
        self.best_val_acc = 0.0
        self.best_val_loss = float('inf')
        self.history = {
            'train_loss': [], 'val_loss': [], 'val_acc': [], 'train_acc': [],
            'val_precision': [], 'val_recall': [], 'val_f1': [], 'val_auc': [],
            'learning_rate': []
        }
        self.early_stopping_counter = 0
        self._resume_from_checkpoint()
    
    def _resume_from_checkpoint(self):
        latest_checkpoint = Path(self.config.system.checkpoint_dir) / 'latest.pth'
        if latest_checkpoint.exists():
            print(f"\n🤖     Found existing checkpoint: {latest_checkpoint}")
            checkpoint = torch.load(latest_checkpoint, map_location=self.device, weights_only=False)
            self.model.load_state_dict(checkpoint['model_state_dict'])
            self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            if self.scheduler and 'scheduler_state_dict' in checkpoint:
                if checkpoint['scheduler_state_dict'] is not None:
                    self.scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
            self.current_epoch = checkpoint['epoch'] + 1
            self.best_val_acc = checkpoint.get('best_val_acc', 0.0)
            self.best_val_loss = checkpoint.get('best_val_loss', float('inf'))
            self.history = checkpoint.get('history', self.history)
            self.early_stopping_counter = checkpoint.get('early_stopping_counter', 0)
            print(f"✅    Resumed from epoch {self.current_epoch}")
            print(f"  Best validation accuracy: {self.best_val_acc:.2f}%")
        else:
            print("\n🤖     No checkpoint found. Starting fresh training...")
    
    def _calculate_class_weights(self) -> torch.Tensor:
        self.config.model.num_classes = len(self.train_loader.dataset.dataset.classes)
        class_counts = torch.zeros(self.config.model.num_classes)
        dataset = self.train_loader.dataset
        
        if hasattr(dataset, 'indices'):
            base_dataset = dataset.dataset
            for idx in dataset.indices:
                _, _, label, _ = base_dataset[idx]
                class_counts[label] += 1
        else:
            for i in range(len(dataset)):
                _, _, label, _ = dataset[i]
                class_counts[label] += 1
        
        class_counts = class_counts.clamp(min=1)
        weights = 1.0 / class_counts
        weights = weights / weights.sum() * len(weights)
        return weights.to(self.device)
    
    def train_epoch(self) -> float:
        self.model.train()
        total_loss = 0.0
        pbar = tqdm(self.train_loader, desc=f'Epoch {self.current_epoch + 1}')
        
        for images, masks, labels, original_sizes in pbar:
            images = images.to(self.device)
            labels = labels.to(self.device)
            
            # Skip augmentation if batch size is too small for mixup/cutmix
            batch_size = images.size(0)
            use_augmentation = (np.random.rand() < self.config.training.augmentation_prob) and (batch_size > 1)
            
            if self.config.training.use_mixup and use_augmentation and batch_size > 1:
                mixed_images, labels_a, labels_b, lam = mixup_data(
                    images, labels, alpha=self.config.training.mixup_alpha
                )
                outputs = self.model(mixed_images)
                cls_loss_a = F.cross_entropy(outputs['logits'], labels_a)
                cls_loss_b = F.cross_entropy(outputs['logits'], labels_b)
                cls_loss = lam * cls_loss_a + (1 - lam) * cls_loss_b
                losses = self.criterion(outputs, (images, masks, labels))
                losses['classification'] = cls_loss
                losses['total'] = (cls_loss + losses['dice'] + losses['bce'])
            elif self.config.training.use_cutmix and use_augmentation and batch_size > 1:
                mixed_images, labels_a, labels_b, lam = cutmix_data(
                    images, labels, alpha=self.config.training.cutmix_alpha
                )
                outputs = self.model(mixed_images)
                cls_loss_a = F.cross_entropy(outputs['logits'], labels_a)
                cls_loss_b = F.cross_entropy(outputs['logits'], labels_b)
                cls_loss = lam * cls_loss_a + (1 - lam) * cls_loss_b
                losses = self.criterion(outputs, (images, masks, labels))
                losses['classification'] = cls_loss
                losses['total'] = (cls_loss + losses['dice'] + losses['bce'])
            else:
                outputs = self.model(images)
                losses = self.criterion(outputs, (images, masks, labels))
            
            self.optimizer.zero_grad()
            losses['total'].backward()
            if self.config.training.gradient_clip > 0:
                nn.utils.clip_grad_norm_(self.model.parameters(), self.config.training.gradient_clip)
            self.optimizer.step()
            
            total_loss += losses['total'].item()
            pbar.set_postfix({'loss': f"{losses['total'].item():.4f}"})
        
        if self.scheduler:
            self.scheduler.step()
        
        return total_loss / max(len(self.train_loader), 1)
    
    @torch.no_grad()
    def _calculate_train_accuracy(self) -> float:
        self.model.eval()
        correct = 0
        total = 0
        max_batches = max(1, len(self.train_loader) // 5)
        for i, (images, masks, labels, _) in enumerate(self.train_loader):
            if i >= max_batches:
                break
            images = images.to(self.device)
            labels = labels.to(self.device)
            outputs = self.model(images)
            _, predicted = outputs['logits'].max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
        self.model.train()
        return 100.0 * correct / total if total > 0 else 0.0
    
    @torch.no_grad()
    def validate(self) -> Tuple[float, float, Dict[str, float]]:
        self.model.eval()
        total_loss = 0.0
        all_preds = []
        all_labels = []
        all_probs = []
        
        for images, masks, labels, original_sizes in self.val_loader:
            images = images.to(self.device)
            labels = labels.to(self.device)
            if isinstance(masks, torch.Tensor):
                masks = masks.to(self.device)
            
            outputs = self.model(images)
            losses = self.criterion(outputs, (images, masks, labels))
            total_loss += losses['total'].item()
            
            logits = outputs['logits']
            probs = F.softmax(logits, dim=1)
            _, predicted = logits.max(1)
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
        
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        all_probs = np.array(all_probs)
        
        avg_loss = total_loss / max(len(self.val_loader), 1)
        accuracy = 100.0 * accuracy_score(all_labels, all_preds)
        precision, recall, f1, _ = precision_recall_fscore_support(
            all_labels, all_preds, average='weighted', zero_division=0
        )
        
        from sklearn.preprocessing import label_binarize
        y_true_bin = label_binarize(all_labels, classes=range(self.config.model.num_classes))
        auc_score = roc_auc_score(y_true_bin, all_probs, average='weighted', multi_class='ovr')
        
        metrics = {
            'precision': precision * 100,
            'recall': recall * 100,
            'f1': f1 * 100,
            'auc': auc_score * 100
        }
        
        return avg_loss, accuracy, metrics
    
    def save_checkpoint(self, is_best: bool = False, epoch_checkpoint: bool = False):
        checkpoint = {
            'epoch': self.current_epoch,
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'scheduler_state_dict': self.scheduler.state_dict() if self.scheduler else None,
            'best_val_acc': self.best_val_acc,
            'best_val_loss': self.best_val_loss,
            'history': self.history,
            'early_stopping_counter': self.early_stopping_counter,
            'config': asdict(self.config)
        }
        
        latest_path = Path(self.config.system.checkpoint_dir) / 'latest.pth'
        torch.save(checkpoint, latest_path)
        
        if is_best:
            best_path = Path(self.config.system.checkpoint_dir) / 'best.pth'
            torch.save(checkpoint, best_path)
            print(f"    ✅   ✓ Saved best checkpoint: {best_path}")
        
        if epoch_checkpoint and (self.current_epoch + 1) % self.config.system.save_frequency == 0:
            epoch_path = Path(self.config.system.checkpoint_dir) / f'epoch_{self.current_epoch + 1:03d}.pth'
            torch.save(checkpoint, epoch_path)
            print(f"    ✅   ✓ Saved epoch checkpoint: {epoch_path}")
    
    def load_checkpoint(self, checkpoint_path: str, load_optimizer: bool = True):
        if not Path(checkpoint_path).exists():
            raise FileNotFoundError(f"Checkpoint not found: {checkpoint_path}")
        
        print(f"\n🤖     Loading checkpoint from: {checkpoint_path}")
        checkpoint = torch.load(checkpoint_path, map_location=self.device, weights_only=False)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        print("✅    Model weights loaded")
        
        if load_optimizer and 'optimizer_state_dict' in checkpoint:
            self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
            print("✅    Optimizer state loaded")
        
        if load_optimizer and self.scheduler and 'scheduler_state_dict' in checkpoint:
            if checkpoint['scheduler_state_dict'] is not None:
                self.scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
                print("✅    Scheduler state loaded")
        
        self.current_epoch = checkpoint.get('epoch', 0) + 1
        self.best_val_acc = checkpoint.get('best_val_acc', 0.0)
        self.best_val_loss = checkpoint.get('best_val_loss', float('inf'))
        self.history = checkpoint.get('history', self.history)
        self.early_stopping_counter = checkpoint.get('early_stopping_counter', 0)
        
        print(f"✅    Training state loaded (epoch {self.current_epoch})")
        print(f"  Best val accuracy: {self.best_val_acc:.2f}%")
        print(f"  Best val loss: {self.best_val_loss:.4f}\n")
    
    def train(self):
        start_epoch = self.current_epoch
        total_epochs = self.config.training.epochs
        
        print(f"🤖     Starting training...")
        print(f"   Device: {self.device}")
        print(f"   Model parameters: {sum(p.numel() for p in self.model.parameters()):,}")
        print(f"   Training from epoch {start_epoch + 1} to {total_epochs}")
        
        for epoch in range(start_epoch, total_epochs):
            self.current_epoch = epoch
            train_loss = self.train_epoch()
            train_acc = self._calculate_train_accuracy()
            val_loss, val_acc, val_metrics = self.validate()
            
            self.history['train_loss'].append(train_loss)
            self.history['train_acc'].append(train_acc)
            self.history['val_loss'].append(val_loss)
            self.history['val_acc'].append(val_acc)
            self.history['val_precision'].append(val_metrics['precision'])
            self.history['val_recall'].append(val_metrics['recall'])
            self.history['val_f1'].append(val_metrics['f1'])
            self.history['val_auc'].append(val_metrics['auc'])
            self.history['learning_rate'].append(self.optimizer.param_groups[0]['lr'])
            
            print(f"\n📊     Epoch {epoch + 1}/{total_epochs}")
            print(f"   Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
            print(f"   Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
            print(f"   Val Precision: {val_metrics['precision']:.2f}%, Recall: {val_metrics['recall']:.2f}%, F1: {val_metrics['f1']:.2f}%")
            print(f"   Val AUC: {val_metrics['auc']:.2f}%, LR: {self.optimizer.param_groups[0]['lr']:.6f}")
            
            is_best = val_acc > self.best_val_acc
            if is_best:
                self.best_val_acc = val_acc
                self.best_val_loss = val_loss
                self.early_stopping_counter = 0
                print(f"   ✅    New best model! Accuracy: {val_acc:.2f}%")
            else:
                self.early_stopping_counter += 1
            
            self.save_checkpoint(is_best=is_best, epoch_checkpoint=True)
            
            if self.early_stopping_counter >= self.config.training.early_stopping_patience:
                print(f"\n✋  ⏹️    Early stopping triggered after {epoch + 1} epochs")
                print(f"   No improvement for {self.early_stopping_counter} consecutive epochs")
                break
        
        print(f"\n🎉     Training complete!")
        print(f"   Best validation accuracy: {self.best_val_acc:.2f}%")
        print(f"   Best validation loss: {self.best_val_loss:.4f}")
        self.plot_history()
    
    def plot_history(self):
        sns.set_style("whitegrid")
        plt.rcParams['figure.dpi'] = 150
        plt.rcParams['savefig.dpi'] = 300
        plt.rcParams['font.size'] = 10
        
        if not self.history['val_acc']:
            print("⚠️    No training history to plot.")
            return
        
        epochs = range(1, len(self.history['val_acc']) + 1)
        print("\n📊     Generating comprehensive training visualizations...")
        
        plt.figure(figsize=(10, 6))
        plt.plot(epochs, self.history['train_acc'], 'o-', linewidth=2.5, markersize=6,
                label='Training Accuracy', color='#2E86AB')
        plt.plot(epochs, self.history['val_acc'], 's-', linewidth=2.5, markersize=6,
                label='Validation Accuracy', color='#A23B72')
        plt.xlabel('Epochs', fontsize=13, fontweight='bold')
        plt.ylabel('Accuracy (%)', fontsize=13, fontweight='bold')
        plt.title('Training vs Validation Accuracy', fontsize=15, fontweight='bold', pad=15)
        plt.legend(loc='lower right', fontsize=11, framealpha=0.95)
        plt.grid(True, alpha=0.3, linestyle='--')
        plt.xlim(left=min(epochs), right=max(epochs))
        
        best_epoch = int(np.argmax(self.history['val_acc'])) + 1
        best_val_acc = max(self.history['val_acc'])
        plt.plot(best_epoch, best_val_acc, 'r*', markersize=15, label=f'Best Val Acc: {best_val_acc:.2f}%')
        plt.legend(loc='lower right', fontsize=11, framealpha=0.95)
        
        plt.tight_layout()
        save_path = Path(self.config.system.log_dir) / 'train_val_accuracy_comparison.png'
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"   ✅    Training vs Validation Accuracy curve saved to: {save_path}")
        
        fig, ax = plt.subplots(figsize=(12, 6))
        ax.plot(epochs, self.history['val_acc'], 'o-', linewidth=2, markersize=6,
                label='Accuracy (%)', color='#FF6B35')
        ax.plot(epochs, self.history['val_precision'], 's-', linewidth=2, markersize=6,
                label='Precision (%)', color='#FF9F1C')
        ax.plot(epochs, self.history['val_recall'], '^-', linewidth=2, markersize=6,
                label='Recall (%)', color='#E71D36')
        ax.plot(epochs, self.history['val_f1'], 'd-', linewidth=2, markersize=6,
                label='F1-Score (%)', color='#C3073F')
        ax.plot(epochs, self.history['val_auc'], 'p-', linewidth=2.5, markersize=6,
                label='AUC Score (%)', color='#00A6FB')
        ax.set_xlabel('Epochs', fontsize=14, fontweight='bold')
        ax.set_ylabel('Percentage (%)', fontsize=14, fontweight='bold')
        ax.set_title('Performance Metrics Over Epochs', fontsize=16, fontweight='bold', pad=20)
        ax.legend(loc='lower right', fontsize=11, framealpha=0.95)
        ax.grid(True, alpha=0.3, linestyle='--')
        ax.set_xlim(left=min(epochs), right=max(epochs))
        plt.tight_layout()
        save_path = Path(self.config.system.log_dir) / 'performance_metrics_over_epochs.png'
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"   ✅    Performance Metrics Over Epochs saved to: {save_path}")
        
        plt.figure(figsize=(10, 6))
        plt.plot(epochs, self.history['train_loss'], 'o-', linewidth=2.5, markersize=6,
                label='Training Loss', color='#06A77D')
        plt.plot(epochs, self.history['val_loss'], 's-', linewidth=2.5, markersize=6,
                label='Validation Loss', color='#F77F00')
        plt.xlabel('Epochs', fontsize=13, fontweight='bold')
        plt.ylabel('Loss', fontsize=13, fontweight='bold')
        plt.title('Training and Validation Loss Curves', fontsize=15, fontweight='bold', pad=15)
        plt.legend(loc='upper right', fontsize=11, framealpha=0.95)
        plt.grid(True, alpha=0.3, linestyle='--')
        plt.xlim(left=min(epochs), right=max(epochs))
        plt.tight_layout()
        save_path = Path(self.config.system.log_dir) / 'loss_curves.png'
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"   ✅    Loss curves saved to: {save_path}")
        
        print(f"\n✅    All training visualizations saved to: {self.config.system.log_dir}\n")

# ============================================================================
# EVALUATOR CLASS WITH COMPONENT IMPORTANCE VISUALIZATION
# ============================================================================

class CAMDFNetEvaluator:
    def __init__(self, model: CAMDFNet, config: CAMDFNetConfig):
        self.model = model
        self.config = config
        self.device = torch.device(config.system.device if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device)
        self.model.eval()
        self.mask_generator = MaskGenerator()
        self.output_dir = Path(config.system.output_dir)
        self.output_dir.mkdir(parents=True, exist_ok=True)
        (self.output_dir / 'summary').mkdir(exist_ok=True)
        (self.output_dir / 'summary/curves').mkdir(exist_ok=True)
        (self.output_dir / 'summary/matrices').mkdir(exist_ok=True)
        (self.output_dir / 'per_image_results').mkdir(exist_ok=True)
        (self.output_dir / 'component_importance').mkdir(exist_ok=True)
        (self.output_dir / 'batch_analysis').mkdir(exist_ok=True)
        plt.style.use('seaborn-v0_8-darkgrid')
        sns.set_palette("husl")
    
    def _safe_resize_2d(self, data_2d: np.ndarray, target_wh: Tuple[int, int]) -> np.ndarray:
        if data_2d.size == 0:
            return np.zeros((target_wh[1], target_wh[0]), dtype=np.float32)
        return cv2.resize(data_2d.astype(np.float32), target_wh, interpolation=cv2.INTER_CUBIC)
    
    def _normalize_for_display(self, arr: np.ndarray) -> np.ndarray:
        mn, mx = arr.min(), arr.max()
        if mx - mn < 1e-8:
            return np.zeros_like(arr, dtype=np.float32)
        return (arr - mn) / (mx - mn)
    
    def create_comprehensive_image_folder_structure(self, image_name: str) -> Dict[str, Path]:
        base = self.output_dir / 'per_image_results' / image_name
        folders = {
            'root': base,
            '0_original': base / '0_original',
            '1_inputs': base / '1_inputs',
            '2_preprocessing': base / '2_preprocessing',
            '2a_spectral': base / '2_preprocessing/spectral',
            '2b_normalized': base / '2_preprocessing/normalized',
            '3_cnn_activations': base / '3_cnn_activations',
            '3a_cnn_layerwise': base / '3_cnn_activations/layerwise',
            '3b_cnn_feature_maps': base / '3_cnn_activations/feature_maps',
            '3c_cnn_gradients': base / '3_cnn_activations/gradients',
            '3d_cnn_statistics': base / '3_cnn_activations/statistics',
            '4_vit_activations': base / '4_vit_activations',
            '4a_vit_self_attention': base / '4_vit_activations/self_attention',
            '4b_vit_headwise': base / '4_vit_activations/headwise',
            '4c_vit_spatial': base / '4_vit_activations/spatial',
            '4d_vit_sequence': base / '4_vit_activations/sequence',
            '4e_vit_statistics': base / '4_vit_activations/statistics',
            '5_fusion_activations': base / '5_fusion_activations',
            '5a_cross_attention': base / '5_fusion_activations/cross_attention',
            '5b_fused_features': base / '5_fusion_activations/fused_features',
            '5c_gated_features': base / '5_fusion_activations/gated',
            '6_segmentation_activations': base / '6_segmentation_activations',
            '6a_pyramid_pooling': base / '6_segmentation_activations/pyramid_pool',
            '6b_attention_refine': base / '6_segmentation_activations/attention_refine',
            '6c_mask_generation': base / '6_segmentation_activations/mask_generation',
            '6d_segmentation_layers': base / '6_segmentation_activations/layers',
            '7_classification_activations': base / '7_classification_activations',
            '7a_classifier_features': base / '7_classification_activations/features',
            '7b_uncertainty': base / '7_classification_activations/uncertainty',
            '7c_probability_dist': base / '7_classification_activations/probabilities',
            '8_masks': base / '8_masks',
            '8a_predicted_masks': base / '8_masks/predicted',
            '8b_ground_truth': base / '8_masks/ground_truth',
            '8c_overlays': base / '8_masks/overlays',
            '8d_comparisons': base / '8_masks/comparisons',
            '9_feature_analysis': base / '9_feature_analysis',
            '9a_feature_maps': base / '9_feature_analysis/feature_maps',
            '9b_heatmaps': base / '9_feature_analysis/heatmaps',
            '9c_pca_analysis': base / '9_feature_analysis/pca',
            '9d_tsne_analysis': base / '9_feature_analysis/tsne',
            '9e_correlations': base / '9_feature_analysis/correlations',
            '10_comprehensive_grids': base / '10_comprehensive_grids',
            '10a_activation_grids': base / '10_comprehensive_grids/activations',
            '10b_feature_grids': base / '10_comprehensive_grids/features',
            '10c_attention_grids': base / '10_comprehensive_grids/attention',
            '10d_final_summary': base / '10_comprehensive_grids/summary',
            '11_statistics': base / '11_statistics',
            '11a_activation_stats': base / '11_statistics/activations',
            '11b_gradient_stats': base / '11_statistics/gradients',
            '11c_performance_metrics': base / '11_statistics/performance',
            '11d_disease_severity': base / '11_statistics/severity',
            '12_debug': base / '12_debug',
            '12a_intermediate_tensors': base / '12_debug/tensors',
            '12b_gradient_flow': base / '12_debug/gradients',
            '12c_model_states': base / '12_debug/states',
            '13_metadata': base / '13_metadata',
            '13a_configurations': base / '13_metadata/configs',
            '13b_predictions': base / '13_metadata/predictions',
            '13c_analysis_reports': base / '13_metadata/reports',
            '13d_visualization_logs': base / '13_metadata/logs',
        }
        for fn, fp in folders.items():
            fp.mkdir(parents=True, exist_ok=True)
            if 'layerwise' in fn or ('layers' in fn and 'segmentation' in str(fp)):
                for i in range(10):
                    (fp / f'layer_{i:03d}').mkdir(exist_ok=True)
            if 'headwise' in fn:
                for i in range(12):
                    (fp / f'head_{i:02d}').mkdir(exist_ok=True)
        return folders

    @torch.no_grad()
    def evaluate(self, dataloader: DataLoader) -> Dict[str, Any]:
        all_preds, all_labels, all_probs, all_uncertainties = [], [], [], []
        all_masks_pred, all_masks_true = [], []
        all_cross_attentions, all_vit_attentions = [], []
        all_spectral_images, all_fused_features = [], []
        
        for images, masks, labels, original_sizes in tqdm(dataloader, desc='Evaluating'):
            images = images.to(self.device)
            labels = labels.to(self.device)
            outputs = self.model(images)
            vit_attentions = self.model.get_vit_self_attention(images)
            logits = outputs['logits']
            probs = F.softmax(logits, dim=1)
            _, predicted = logits.max(1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probs.cpu().numpy())
            all_uncertainties.extend(outputs['uncertainty'].cpu().numpy())
            
            batch_size = images.size(0)
            for i in range(batch_size):
                orig_w, orig_h = original_sizes[i]
                mask_pred_i = outputs['mask'][i:i+1]
                mask_pred_resized = F.interpolate(
                    mask_pred_i, size=(int(orig_h), int(orig_w)),
                    mode='bilinear', align_corners=True
                )
                all_masks_pred.append(torch.sigmoid(mask_pred_resized).cpu().numpy()[0])
                if isinstance(masks, torch.Tensor):
                    mt = masks[i].cpu().numpy()
                else:
                    mt = masks[i].numpy() if isinstance(masks[i], torch.Tensor) else masks[i]
                if mt.ndim == 2:
                    mt = mt[np.newaxis, ...]
                all_masks_true.append(mt)
            
            cross_attn = outputs['cross_attention_weights']
            if isinstance(cross_attn, tuple):
                cross_attn = cross_attn[0]
            all_cross_attentions.append(cross_attn.cpu().numpy())
            all_vit_attentions.append({k: v.cpu().numpy() for k, v in vit_attentions.items()})
            all_spectral_images.extend(outputs['spectral_image'].cpu().numpy())
            all_fused_features.extend(outputs['fused_features'].cpu().numpy())
        
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        all_probs = np.array(all_probs)
        all_uncertainties = np.array(all_uncertainties)
        
        if all_cross_attentions:
            all_cross_attentions = np.concatenate(all_cross_attentions, axis=0)
        
        accuracy = accuracy_score(all_labels, all_preds)
        precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted', zero_division=0)
        
        iou_scores, dice_scores = [], []
        for mp, mt in zip(all_masks_pred, all_masks_true):
            mps, mts = mp.squeeze(), mt.squeeze()
            if mps.shape != mts.shape:
                mps = cv2.resize(mps, (mts.shape[1], mts.shape[0]), interpolation=cv2.INTER_LINEAR)
            mpb = (mps > 0.5).astype(np.float32).flatten()
            mtf = mts.flatten().astype(np.float32)
            inter = (mpb * mtf).sum()
            union = mpb.sum() + mtf.sum() - inter
            iou_scores.append(inter / (union + 1e-8))
            dice_scores.append((2 * inter) / (mpb.sum() + mtf.sum() + 1e-8))
        
        return {
            'accuracy': accuracy, 'precision': precision, 'recall': recall,
            'f1_score': f1, 'iou': np.mean(iou_scores) if iou_scores else 0.0,
            'dice': np.mean(dice_scores) if dice_scores else 0.0,
            'predictions': all_preds, 'labels': all_labels,
            'probabilities': all_probs, 'uncertainties': all_uncertainties,
            'masks_pred': all_masks_pred, 'masks_true': all_masks_true,
            'cross_attentions': all_cross_attentions,
            'vit_attentions': all_vit_attentions,
            'spectral_images': np.array(all_spectral_images),
            'fused_features': np.array(all_fused_features)
        }
    
    def visualize_activation_layer(self, activation, layer_name, folder, original_size, cmap='viridis'):
        if activation is None: return
        anp = activation.squeeze().cpu().float().numpy()
        tw = (int(original_size[0]), int(original_size[1]))
        
        if anp.ndim == 3:
            nc = min(anp.shape[0], 16)
            cols, rows = 4, (nc + 3) // 4
            fig, axes = plt.subplots(rows, cols, figsize=(cols*4, rows*3))
            if rows == 1 and cols == 1: axes = np.array([[axes]])
            elif rows == 1: axes = axes.reshape(1, -1)
            elif cols == 1: axes = axes.reshape(-1, 1)
            for idx in range(nc):
                r, c = idx // cols, idx % cols
                cr = self._safe_resize_2d(anp[idx], tw)
                cn = self._normalize_for_display(cr)
                im = axes[r, c].imshow(cn, cmap=cmap, aspect='auto')
                axes[r, c].set_title(f'Ch {idx}', fontsize=8, fontweight='bold'); axes[r, c].axis('off')
                if idx == 0: plt.colorbar(im, ax=axes[r, c], fraction=0.046, pad=0.04)
            for idx in range(nc, rows*cols): axes[idx//cols, idx%cols].axis('off')
            plt.suptitle(f'{layer_name} - Feature Maps', fontsize=12, fontweight='bold', y=0.98)
            plt.tight_layout()
            plt.savefig(str(folder / f'{layer_name}_feature_grid.png'), dpi=300, bbox_inches='tight', facecolor='white')
            plt.close()
            for idx in range(min(nc, 8)):
                cn = self._normalize_for_display(self._safe_resize_2d(anp[idx], tw))
                plt.imsave(str(folder / f'{layer_name}_channel_{idx:03d}.png'), cn, cmap=cmap)
        elif anp.ndim == 2:
            ar = self._safe_resize_2d(anp, tw)
            an = self._normalize_for_display(ar)
            fig, (a1, a2) = plt.subplots(1, 2, figsize=(12, 5))
            im1 = a1.imshow(an, cmap=cmap); a1.set_title(f'{layer_name}', fontsize=10, fontweight='bold'); a1.axis('off')
            plt.colorbar(im1, ax=a1, fraction=0.046, pad=0.04)
            a2.imshow(an, cmap=cmap, alpha=0.8); a2.contour(an, levels=10, colors='white', alpha=0.5, linewidths=0.5)
            a2.set_title(f'{layer_name} - Contours', fontsize=10, fontweight='bold'); a2.axis('off')
            plt.tight_layout()
            plt.savefig(str(folder / f'{layer_name}_visualization.png'), dpi=300, bbox_inches='tight', facecolor='white')
            plt.close()
            plt.imsave(str(folder / f'{layer_name}_simple.png'), an, cmap=cmap)
        elif anp.ndim == 1 and len(anp) > 0:
            fig, ax = plt.subplots(figsize=(10, 4))
            ax.bar(range(len(anp)), anp, color='skyblue', edgecolor='black', alpha=0.7)
            ax.set_title(f'{layer_name}', fontsize=11, fontweight='bold'); ax.grid(True, alpha=0.3, axis='y')
            plt.tight_layout()
            plt.savefig(str(folder / f'{layer_name}_values.png'), dpi=300, bbox_inches='tight', facecolor='white')
            plt.close()
    
    def visualize_activation_statistics(self, activation, layer_name, folder):
        if activation is None: return
        anp = activation.cpu().float().numpy().flatten()
        if len(anp) < 2: return
        fig, ((a1,a2),(a3,a4)) = plt.subplots(2, 2, figsize=(12, 10))
        a1.hist(anp, bins=min(100, len(anp)), color='skyblue', edgecolor='black', alpha=0.7)
        a1.set_title('Distribution', fontsize=11, fontweight='bold'); a1.grid(True, alpha=0.3)
        a2.boxplot(anp, vert=True, patch_artist=True, boxprops=dict(facecolor='lightcoral'))
        a2.set_title('Box Plot', fontsize=11, fontweight='bold'); a2.grid(True, alpha=0.3, axis='y')
        scipy_stats.probplot(anp, dist="norm", plot=a3)
        a3.set_title('Q-Q Plot', fontsize=11, fontweight='bold'); a3.grid(True, alpha=0.3)
        parts = a4.violinplot(anp, showmeans=True, showmedians=True)
        for pc in parts['bodies']:
            pc.set_facecolor('lightgreen')
            pc.set_alpha(0.7)
        a4.set_title('Violin Plot', fontsize=11, fontweight='bold'); a4.grid(True, alpha=0.3, axis='y')
        plt.suptitle(f'{layer_name} - Stats', fontsize=13, fontweight='bold', y=0.98)
        plt.tight_layout()
        plt.savefig(str(folder / f'{layer_name}_statistics.png'), dpi=300, bbox_inches='tight', facecolor='white')
        plt.close()
        sd = {'layer_name': layer_name, 'mean': float(np.mean(anp)), 'std': float(np.std(anp)),
              'min': float(np.min(anp)), 'max': float(np.max(anp)), 'median': float(np.median(anp)),
              'skewness': float(scipy_stats.skew(anp)), 'kurtosis': float(scipy_stats.kurtosis(anp)),
              'num_elements': len(anp), 'non_zero_pct': float(np.sum(anp!=0)/len(anp)*100)}
        with open(str(folder / f'{layer_name}_statistics.json'), 'w') as f:
            json.dump(sd, f, indent=2)
    
    def visualize_gradient_flow(self, gradients, folders, image_name):
        if not gradients: return
        gm = {n: float(np.sqrt(np.sum(g.cpu().numpy()**2))) for n, g in gradients.items() if g is not None}
        if not gm: return
        sm = dict(sorted(gm.items(), key=lambda x: x[1], reverse=True))
        fig, (a1, a2) = plt.subplots(1, 2, figsize=(15, 6))
        names = list(sm.keys())[:15]; mags = [sm[n] for n in names]
        a1.bar(range(len(names)), mags, color='lightcoral', edgecolor='darkred')
        a1.set_xticks(range(len(names))); a1.set_xticklabels([n[:20] for n in names], rotation=45, ha='right', fontsize=8)
        a1.set_title('Gradient Magnitudes', fontsize=12, fontweight='bold'); a1.grid(True, alpha=0.3, axis='y')
        ag = np.concatenate([g.cpu().numpy().flatten() for g in gradients.values() if g is not None])
        a2.hist(ag, bins=100, color='lightblue', edgecolor='darkblue', alpha=0.7)
        a2.set_title('Gradient Distribution', fontsize=12, fontweight='bold'); a2.grid(True, alpha=0.3)
        plt.suptitle(f'Gradient Flow: {image_name}', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.savefig(str(folders['3c_cnn_gradients'] / f'{image_name}_gradient_analysis.png'), dpi=300, bbox_inches='tight', facecolor='white')
        plt.close()
    
    def create_comprehensive_activation_report(self, all_activations, folders, image_name, original_size):
        print(f"    📊 Generating comprehensive activation report...")
        cmaps = {'input':'gray','spectral':'viridis','cnn':'plasma','vit':'magma',
                 'attention':'hot','fusion':'coolwarm','segmentation':'summer',
                 'classification':'spring','mask':'bone'}
        
        for aname, atensor in all_activations.items():
            if atensor is None: continue
            nl = aname.lower()
            atype, tfolder = 'other', folders['12_debug']
            if 'input' in nl: atype, tfolder = 'input', folders['1_inputs']
            elif 'spectral' in nl: atype, tfolder = 'spectral', folders['2a_spectral']
            elif 'cnn' in nl:
                atype = 'cnn'
                if 'layer' in aname:
                    ln = aname.split('_')[-1]
                    ln = ln if ln.isdigit() else '0'
                    lf = folders['3a_cnn_layerwise'] / f'layer_{int(ln):03d}'
                    lf.mkdir(exist_ok=True); tfolder = lf
                else: tfolder = folders['3b_cnn_feature_maps']
            elif 'vit' in nl:
                atype = 'vit'
                if 'spatial' in nl: tfolder = folders['4c_vit_spatial']
                elif 'sequence' in nl: tfolder = folders['4d_vit_sequence']
                else: tfolder = folders['4_vit_activations']
            elif 'attention' in nl:
                atype = 'attention'
                if 'cross' in nl: tfolder = folders['5a_cross_attention']
                elif 'self' in nl: tfolder = folders['4a_vit_self_attention']
                else: tfolder = folders['6b_attention_refine']
            elif 'fusion' in nl or 'fused' in nl: atype, tfolder = 'fusion', folders['5_fusion_activations']
            elif 'mask' in nl or 'seg' in nl: atype, tfolder = 'segmentation', folders['6_segmentation_activations']
            elif 'class' in nl or 'logit' in nl: atype, tfolder = 'classification', folders['7_classification_activations']
            elif 'uncertainty' in nl: atype, tfolder = 'classification', folders['7b_uncertainty']
            elif 'pool' in nl: atype, tfolder = 'segmentation', folders['6a_pyramid_pooling']
            elif 'proj' in nl: atype, tfolder = 'cnn', folders['3_cnn_activations']
            elif 'gated' in nl: atype, tfolder = 'fusion', folders['5c_gated_features']
            tfolder.mkdir(parents=True, exist_ok=True)
            
            print(f"      📈 Visualizing {aname}...")
            self.visualize_activation_layer(atensor, aname, tfolder, original_size, cmaps.get(atype, 'viridis'))
            if atensor.numel() < 1000000:
                sf = folders['11a_activation_stats']; sf.mkdir(exist_ok=True)
                self.visualize_activation_statistics(atensor, aname, sf)
                if atensor.numel() < 10000:
                    np.save(str(folders['12a_intermediate_tensors'] / f'{aname}.npy'), atensor.cpu().numpy())
                else:
                    s = {'name': aname, 'shape': list(atensor.shape),
                         'mean': float(atensor.float().mean()), 'std': float(atensor.float().std()),
                         'min': float(atensor.float().min()), 'max': float(atensor.float().max()),
                         'numel': int(atensor.numel())}
                    with open(str(folders['12a_intermediate_tensors'] / f'{aname}_summary.json'), 'w') as f:
                        json.dump(s, f, indent=2)
        
        self.create_activation_summary_grid(all_activations, folders, image_name, original_size)
        self._create_feature_analysis(all_activations, folders, image_name, original_size)
        self._create_prob_uncertainty_viz(all_activations, folders, image_name)
        self._create_comparison_viz(all_activations, folders, image_name, original_size)
    
    def _create_feature_analysis(self, all_activations, folders, image_name, original_size):
        print(f"      📊 Creating feature analysis...")
        kf = {}
        for n, t in all_activations.items():
            if any(kw in n.lower() for kw in ['feature','fused','cnn_aligned','vit_spatial']):
                if t is not None and 1 < t.numel() < 100000:
                    flat = t.cpu().float().numpy().flatten()[:5000]
                    if len(flat) > 1: kf[n] = flat
        if not kf: return
        
        tw = (int(original_size[0]), int(original_size[1]))
        for name, tensor in all_activations.items():
            if tensor is None or tensor.dim() != 4: continue
            if tensor.size(1) >= 3:
                fm = np.transpose(tensor[0,:3].cpu().float().numpy(), (1,2,0))
            else:
                f2d = tensor[0].mean(dim=0).cpu().float().numpy()
                fm = np.stack([f2d]*3, axis=-1)
            fr = cv2.resize(fm, tw, interpolation=cv2.INTER_CUBIC)
            plt.imsave(str(folders['9a_feature_maps'] / f'{name}_feature_map.png'), self._normalize_for_display(fr))
        for name, tensor in all_activations.items():
            if tensor is None or tensor.dim() < 2: continue
            if tensor.dim() == 4: hd = tensor[0].mean(dim=0).cpu().float().numpy()
            elif tensor.dim() == 3: hd = tensor[0].mean(dim=0).cpu().float().numpy() if tensor.size(0)>1 else tensor.mean(dim=0).cpu().float().numpy()
            elif tensor.dim() == 2: hd = tensor.cpu().float().numpy()
            else: continue
            if hd.ndim != 2 or hd.size < 4: continue
            hr = self._safe_resize_2d(hd, tw)
            fig, ax = plt.subplots(figsize=(10, 8))
            im = ax.imshow(hr, cmap='hot', aspect='auto'); ax.set_title(f'{name}', fontsize=12, fontweight='bold'); ax.axis('off')
            plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
            plt.savefig(str(folders['9b_heatmaps'] / f'{name}_heatmap.png'), dpi=300, bbox_inches='tight', facecolor='white'); plt.close()
        
        if len(kf) >= 2:
            from sklearn.decomposition import PCA
            from sklearn.preprocessing import StandardScaler
            fns = list(kf.keys()); fvs = [kf[n] for n in fns]
            ml = max(len(v) for v in fvs)
            fm = np.zeros((len(fvs), ml))
            for i, v in enumerate(fvs): fm[i,:len(v)] = v[:ml]
            nc = min(3, len(fns), fm.shape[1])
            if nc >= 2:
                fs = StandardScaler().fit_transform(fm)
                pca = PCA(n_components=nc); pc = pca.fit_transform(fs); ev = pca.explained_variance_ratio_
                fig, ax = plt.subplots(figsize=(10, 8))
                sc = ax.scatter(pc[:,0], pc[:,1], c=range(len(fns)), cmap='viridis', s=100, alpha=0.7)
                for i, n in enumerate(fns): ax.annotate(n[:15], (pc[i,0], pc[i,1]), fontsize=8)
                ax.set_title(f'PCA: PC1:{ev[0]:.2%}, PC2:{ev[1]:.2%}', fontsize=12, fontweight='bold')
                plt.colorbar(sc, ax=ax); plt.tight_layout()
                plt.savefig(str(folders['9c_pca_analysis'] / f'{image_name}_pca.png'), dpi=300, bbox_inches='tight', facecolor='white'); plt.close()
        
        if len(kf) >= 3:
            from sklearn.manifold import TSNE
            from sklearn.preprocessing import StandardScaler
            fns = list(kf.keys()); fvs = [kf[n] for n in fns]
            ml = max(len(v) for v in fvs)
            fm = np.zeros((len(fvs), ml))
            for i, v in enumerate(fvs): fm[i,:len(v)] = v[:ml]
            perp = min(5, len(fns)-1)
            if perp >= 1:
                fs = StandardScaler().fit_transform(fm)
                tr = TSNE(n_components=2, random_state=42, perplexity=perp, n_iter=300).fit_transform(fs)
                fig, ax = plt.subplots(figsize=(10, 8))
                sc = ax.scatter(tr[:,0], tr[:,1], c=range(len(fns)), cmap='plasma', s=100, alpha=0.7)
                for i, n in enumerate(fns): ax.annotate(n[:15], (tr[i,0], tr[i,1]), fontsize=8)
                ax.set_title(f't-SNE: {image_name}', fontsize=12, fontweight='bold')
                plt.colorbar(sc, ax=ax); plt.grid(True, alpha=0.3); plt.tight_layout()
                plt.savefig(str(folders['9d_tsne_analysis'] / f'{image_name}_tsne.png'), dpi=300, bbox_inches='tight', facecolor='white'); plt.close()
        
        if len(kf) >= 2:
            from scipy.stats import pearsonr
            fns = list(kf.keys()); fvs = [kf[n] for n in fns]
            ml = max(len(v) for v in fvs)
            fm = np.zeros((len(fvs), ml))
            for i, v in enumerate(fvs): fm[i,:len(v)] = v[:ml]
            nf = len(fns); cm = np.zeros((nf, nf))
            for i in range(nf):
                for j in range(nf): cm[i,j], _ = pearsonr(fm[i], fm[j])
            fig, ax = plt.subplots(figsize=(12, 10))
            im = ax.imshow(cm, cmap='coolwarm', vmin=-1, vmax=1)
            ax.set_xticks(range(nf)); ax.set_yticks(range(nf))
            ax.set_xticklabels([n[:15] for n in fns], rotation=45, ha='right', fontsize=8)
            ax.set_yticklabels([n[:15] for n in fns], fontsize=8)
            ax.set_title(f'Correlation: {image_name}', fontsize=14, fontweight='bold')
            plt.colorbar(im, ax=ax); plt.tight_layout()
            plt.savefig(str(folders['9e_correlations'] / f'{image_name}_correlation.png'), dpi=300, bbox_inches='tight', facecolor='white'); plt.close()
    
    def _create_prob_uncertainty_viz(self, all_activations, folders, image_name):
        print(f"      📈 Creating probability/uncertainty visualizations...")
        if 'cls_logits' in all_activations and all_activations['cls_logits'] is not None:
            probs = F.softmax(all_activations['cls_logits'], dim=1).cpu().numpy()[0]
            fig, ax = plt.subplots(figsize=(10, 6))
            bars = ax.bar(range(len(probs)), probs, color='skyblue', edgecolor='black', alpha=0.7)
            ax.set_xlabel('Class'); ax.set_ylabel('Probability'); ax.set_ylim([0, 1.1])
            ax.set_title(f'Probabilities: {image_name}', fontsize=12, fontweight='bold'); ax.grid(True, alpha=0.3, axis='y')
            for b, p in zip(bars, probs): ax.text(b.get_x()+b.get_width()/2., b.get_height()+0.01, f'{p:.3f}', ha='center', fontsize=9)
            plt.tight_layout()
            plt.savefig(str(folders['7c_probability_dist'] / f'{image_name}_probs.png'), dpi=300, bbox_inches='tight', facecolor='white'); plt.close()
        if 'uncertainty' in all_activations and all_activations['uncertainty'] is not None:
            uv = all_activations['uncertainty'].cpu().numpy().flatten()[0]
            fig, ax = plt.subplots(figsize=(8, 6))
            ax.bar(['Uncertainty'], [uv], color='lightblue', alpha=0.7)
            ax.set_ylim([0, 1]); ax.set_title(f'Uncertainty: {uv:.3f}', fontsize=11, fontweight='bold'); ax.grid(True, alpha=0.3, axis='y')
            plt.tight_layout()
            plt.savefig(str(folders['7b_uncertainty'] / f'{image_name}_uncertainty.png'), dpi=300, bbox_inches='tight', facecolor='white'); plt.close()
    
    def _create_comparison_viz(self, all_activations, folders, image_name, original_size):
        print(f"      🔄 Creating comparison visualizations...")
        tw = (int(original_size[0]), int(original_size[1]))
        attn_names = [n for n in all_activations.keys() if 'attention' in n.lower()]
        if len(attn_names) >= 2:
            fig, axes = plt.subplots(1, min(3, len(attn_names)), figsize=(15, 5))
            if not isinstance(axes, np.ndarray): axes = [axes]
            for idx, an in enumerate(attn_names[:len(axes)]):
                t = all_activations[an]
                if t is not None:
                    am = t.squeeze().cpu().float().numpy()
                    if am.ndim == 2:
                        ar = self._safe_resize_2d(am, tw)
                        axes[idx].imshow(ar, cmap='hot'); axes[idx].set_title(an[:20], fontsize=10, fontweight='bold'); axes[idx].axis('off')
            plt.suptitle(f'Attention Comparison: {image_name}', fontsize=12, fontweight='bold')
            plt.tight_layout()
            plt.savefig(str(folders['8d_comparisons'] / f'{image_name}_attn_comparison.png'), dpi=300, bbox_inches='tight', facecolor='white'); plt.close()
    
    def create_activation_summary_grid(self, all_activations, folders, image_name, original_size):
        tw = (int(original_size[0]), int(original_size[1]))
        key_acts = {
            'Input': all_activations.get('input'), 'Spectral': all_activations.get('spectral_output'),
            'CNN Features': all_activations.get('cnn_features_raw'), 'ViT Spatial': all_activations.get('vit_spatial'),
            'Fused': all_activations.get('fused_features'), 'Cross Attn': all_activations.get('cross_attention_output'),
            'Attn Refine': all_activations.get('attention_output'), 'Mask Logits': all_activations.get('mask_logits'),
            'Final Mask': all_activations.get('final_mask'), 'Cls Logits': all_activations.get('cls_logits')
        }
        key_acts = {k: v for k, v in key_acts.items() if v is not None}
        if not key_acts: return
        cols = 4; rows = (len(key_acts) + cols - 1) // cols
        fig, axes = plt.subplots(rows, cols, figsize=(cols*4, rows*3.5))
        if rows == 1 and cols == 1: axes = np.array([[axes]])
        elif rows == 1: axes = axes.reshape(1, -1)
        elif cols == 1: axes = axes.reshape(-1, 1)
        
        for idx, (name, act) in enumerate(key_acts.items()):
            r, c = idx // cols, idx % cols
            anp = act.squeeze().cpu().float().numpy()
            if anp.ndim == 3: anp = anp.mean(axis=0)
            elif anp.ndim == 4: anp = anp.mean(axis=(0, 1))
            elif anp.ndim > 4: anp = anp.mean(axis=tuple(range(anp.ndim - 2)))
            elif anp.ndim < 2:
                axes[r, c].text(0.5, 0.5, f'{name}\n{anp}', ha='center', va='center', fontsize=10)
                axes[r, c].set_title(name, fontsize=9, fontweight='bold'); axes[r, c].axis('off'); continue
            
            ar = self._safe_resize_2d(anp, tw)
            an = self._normalize_for_display(ar)
            cmap = 'gray' if 'mask' in name.lower() else ('hot' if 'attn' in name.lower() else 'viridis')
            axes[r, c].imshow(an, cmap=cmap)
            axes[r, c].set_title(name, fontsize=9, fontweight='bold', pad=3); axes[r, c].axis('off')
        
        for idx in range(len(key_acts), rows*cols): axes[idx//cols, idx%cols].axis('off')
        plt.suptitle(f'Activation Summary: {image_name}', fontsize=14, fontweight='bold', y=0.98)
        plt.tight_layout()
        plt.savefig(str(folders['10a_activation_grids'] / f'{image_name}_activation_summary.png'), dpi=300, bbox_inches='tight', facecolor='white')
        plt.close()
    
    def compute_disease_severity(self, mask: np.ndarray) -> Dict[str, Any]:
        mb = (mask > 127).astype(np.uint8)
        total = mb.size; affected = int(np.sum(mb)); pct = (affected / total) * 100
        nl, li = cv2.connectedComponents(mb)
        num_lesions = nl - 1
        if num_lesions > 0:
            ls = [int(np.sum(li == i)) for i in range(1, nl)]
            avg_ls, max_ls = float(np.mean(ls)), float(np.max(ls))
        else:
            avg_ls, max_ls = 0.0, 0.0
        if pct < 2: sl = "Healthy"
        elif pct < 5: sl = "Minimal"
        elif pct < 15: sl = "Mild"
        elif pct < 30: sl = "Moderate"
        elif pct < 50: sl = "Severe"
        else: sl = "Critical"
        return {'affected_percentage': pct, 'severity_level': sl, 'num_lesions': num_lesions,
                'avg_lesion_size': avg_ls, 'max_lesion_size': max_ls,
                'total_affected_pixels': affected, 'total_pixels': total}
    
    def generate_recommendation(self, severity, confidence):
        sl = severity['severity_level']
        if sl == "Critical" and confidence > 0.8: return "🚨 URGENT: Immediate treatment required."
        elif sl == "Severe": return "⚠️ Apply fungicide immediately."
        elif sl == "Moderate": return "📋 Treat within 48 hours."
        elif sl == "Mild": return "👀 Monitor. Consider preventive treatment."
        elif sl == "Minimal": return "✅ Minor issue. Monitor."
        else: return "🌿 HEALTHY: No action needed."
    
    def create_ultimate_comprehensive_grid(self, image_name, original_img, pred_mask, gt_mask,
                                          all_activations, pred_class, class_name, confidence,
                                          severity, folders, original_size):
        tw = (int(original_size[0]), int(original_size[1]))
        fig = plt.figure(figsize=(28, 20))
        gs = gridspec.GridSpec(6, 6, hspace=0.4, wspace=0.4)
        
        ax1 = fig.add_subplot(gs[0, 0]); ax1.imshow(original_img)
        ax1.set_title(f'Original {tw[0]}x{tw[1]}', fontsize=10, fontweight='bold'); ax1.axis('off')
        
        ax2 = fig.add_subplot(gs[0, 1]); ax2.imshow(pred_mask, cmap='gray')
        ax2.set_title(f'Pred Mask\n{severity["affected_percentage"]:.1f}%', fontsize=10, fontweight='bold'); ax2.axis('off')
        
        ax3 = fig.add_subplot(gs[0, 2])
        if gt_mask is not None:
            gmu = (gt_mask * 255).astype(np.uint8) if gt_mask.max() <= 1 else gt_mask.astype(np.uint8)
            ax3.imshow(gmu, cmap='gray'); ax3.set_title('GT Mask', fontsize=10, fontweight='bold')
        else:
            ax3.text(0.5, 0.5, 'No GT', ha='center', va='center', fontsize=12)
            ax3.set_title('GT Mask', fontsize=10, fontweight='bold')
        ax3.axis('off')
        
        ax4 = fig.add_subplot(gs[0, 3])
        mc = cv2.applyColorMap(pred_mask, cv2.COLORMAP_JET)
        ov = cv2.addWeighted(original_img, 0.6, mc, 0.4, 0)
        ax4.imshow(ov); ax4.set_title('Pred Overlay', fontsize=10, fontweight='bold'); ax4.axis('off')
        
        ax5 = fig.add_subplot(gs[0, 4])
        if gt_mask is not None:
            gmu = (gt_mask * 255).astype(np.uint8) if gt_mask.max() <= 1 else gt_mask.astype(np.uint8)
            gc = cv2.applyColorMap(gmu, cv2.COLORMAP_JET)
            if original_img.shape[:2] != gc.shape[:2]:
                gc = cv2.resize(gc, (original_img.shape[1], original_img.shape[0]))
            ax5.imshow(cv2.addWeighted(original_img, 0.6, gc, 0.4, 0))
            ax5.set_title('GT Overlay', fontsize=10, fontweight='bold')
        else:
            ax5.text(0.5, 0.5, 'No GT', ha='center', va='center')
            ax5.set_title('GT Overlay', fontsize=10, fontweight='bold')
        ax5.axis('off')
        
        ax6 = fig.add_subplot(gs[0, 5])
        st = f"⚕️ SEVERITY\n\nLevel: {severity['severity_level']}\nAffected: {severity['affected_percentage']:.1f}%\nLesions: {severity['num_lesions']}"
        ax6.text(0.1, 0.5, st, fontsize=11, va='center', bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8))
        ax6.set_title('Severity', fontsize=10, fontweight='bold'); ax6.axis('off')
        
        for row_idx, (prefix, cmap, label) in enumerate(
            [('cnn', 'plasma', 'CNN'), ('vit', 'magma', 'ViT'), ('attention', 'hot', 'Attn/Fusion')], start=1):
            keys = [k for k in all_activations.keys() if prefix in k.lower()]
            if prefix == 'attention':
                keys += [k for k in all_activations.keys() if 'fused' in k.lower()]
            for idx, aname in enumerate(keys[:3]):
                ax = fig.add_subplot(gs[row_idx, idx*2:(idx*2)+2])
                act = all_activations[aname]
                if act is not None:
                    anp = act.squeeze().cpu().float().numpy()
                    if anp.ndim == 3: anp = anp.mean(axis=0)
                    elif anp.ndim < 2: ax.text(0.5, 0.5, f'{aname[:20]}', ha='center', va='center'); ax.axis('off'); continue
                    ar = self._safe_resize_2d(anp, tw)
                    ax.imshow(self._normalize_for_display(ar), cmap=cmap)
                    ax.set_title(f'{label}: {aname[:20]}', fontsize=9, fontweight='bold'); ax.axis('off')
        
        seg_keys = [k for k in all_activations.keys() if any(w in k.lower() for w in ['mask','seg','pool'])]
        for idx, aname in enumerate(seg_keys[:3]):
            ax = fig.add_subplot(gs[4, idx*2:(idx*2)+2])
            act = all_activations[aname]
            if act is not None:
                anp = act.squeeze().cpu().float().numpy()
                if anp.ndim == 3: anp = anp.mean(axis=0)
                elif anp.ndim < 2: ax.text(0.5, 0.5, f'{aname[:20]}', ha='center', va='center'); ax.axis('off'); continue
                ax.imshow(self._normalize_for_display(self._safe_resize_2d(anp, tw)), cmap='summer')
                ax.set_title(f'Seg: {aname[:20]}', fontsize=9, fontweight='bold'); ax.axis('off')
        
        ax_s = fig.add_subplot(gs[5, :])
        summary = (f"🎯 CAMDF-NET  REPORT\nImage: {image_name} | Size: {tw[0]}x{tw[1]}\n"
                   f"Predicted: {class_name} ({confidence:.2%}) | Severity: {severity['severity_level']} ({severity['affected_percentage']:.1f}%)\n"
                   f"Lesions: {severity['num_lesions']} | Activations: {len(all_activations)}\n"
                   f"Recommendation: {self.generate_recommendation(severity, confidence)}")
        ax_s.text(0.1, 0.5, summary, fontsize=12, va='center', bbox=dict(boxstyle='round', facecolor='lightcoral', alpha=0.8))
        ax_s.set_title('DIAGNOSIS SUMMARY', fontsize=14, fontweight='bold'); ax_s.axis('off')
        
        plt.suptitle(f'CAMDF-Net  - COMPREHENSIVE: {image_name}', fontsize=16, fontweight='bold', y=0.98)
        plt.tight_layout()
        sp = str(folders['10d_final_summary'] / f'{image_name}_ULTIMATE_grid.png')
        plt.savefig(sp, dpi=300, bbox_inches='tight', facecolor='white'); plt.close()
        return sp
    
    def count_visualizations(self, folder_path):
        return sum(1 for _ in folder_path.rglob('*.png')) + sum(1 for _ in folder_path.rglob('*.jpg'))
    
    def generate_visualization_summary(self, folders, image_name, saved_paths):
        summary = {'image_name': image_name, 'timestamp': datetime.datetime.now().isoformat(),
                   'total_visualizations': self.count_visualizations(folders['root']),
                   'folders': {}, 'key_files': saved_paths}
        for fn, fp in folders.items():
            if fp.exists():
                total = sum(len(list(fp.rglob(f'*.{ext}'))) for ext in ['png','jpg','npy','json','txt'])
                if total > 0:
                    summary['folders'][fn] = {'total': total}
        with open(str(folders['13_metadata'] / f'{image_name}_summary.json'), 'w') as f:
            json.dump(summary, f, indent=2)
        html = f"""<!DOCTYPE html><html><head><title>CAMDF-Net Report: {image_name}</title>
        <style>body{{font-family:Arial;margin:40px;background:#f5f5f5}}.header{{background:#2c3e50;color:white;padding:20px;border-radius:10px}}
        .section{{background:white;padding:20px;margin:20px 0;border-radius:10px;box-shadow:0 2px 5px rgba(0,0,0,0.1)}}</style></head>
        <body><div class="header"><h1>CAMDF-Net  Report: {image_name}</h1>
        <p>Generated: {datetime.datetime.now().strftime('%Y-%m-d %H:%M:%S')}</p></div>
        <div class="section"><h3>Summary</h3><p>Total Visualizations: {summary['total_visualizations']}</p>
        <p>Folders: {len(summary['folders'])}</p></div></body></html>"""
        with open(str(folders['13_metadata'] / f'{image_name}_report.html'), 'w') as f:
            f.write(html)
        print(f"    📋 Summary saved")
    
    def visualize_component_importance(self, image_path, class_names, ground_truth_label=None):
        print(f"\n🔬 Visualizing Component Importance for: {Path(image_path).name}")
        
        original_image = Image.open(image_path).convert('RGB')
        original_size = original_image.size
        original_img_np = np.array(original_image)
        image_name = Path(image_path).stem
        
        transform = transforms.Compose([
            transforms.Resize((self.config.dataset.image_size, self.config.dataset.image_size)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        image_tensor = transform(original_image).unsqueeze(0).to(self.device)
        
        with torch.no_grad():
            outputs = self.model(image_tensor)
            all_activations = self.model.get_all_activations()
        
        probs = F.softmax(outputs['logits'], dim=1).cpu().numpy()[0]
        pred_class = int(np.argmax(probs))
        confidence = float(probs[pred_class])
        pred_class_name = class_names[pred_class]
        
        if ground_truth_label is not None:
            gt_class_name = class_names[ground_truth_label]
            gt_correct = (ground_truth_label == pred_class)
        else:
            gt_class_name = "Unknown"
            gt_correct = None
        
        spectral_image = outputs['spectral_image'].cpu().numpy()[0].transpose(1, 2, 0)
        vit_attention = all_activations.get('vit_spatial', None)
        cnn_features = all_activations.get('cnn_features_raw', None)
        
        fig = plt.figure(figsize=(20, 12))
        gs = gridspec.GridSpec(3, 4, hspace=0.3, wspace=0.3, width_ratios=[1, 1, 1, 0.8])
        
        ax1 = fig.add_subplot(gs[0, 0])
        ax1.imshow(original_img_np)
        ax1.set_title('Original Image', fontsize=12, fontweight='bold')
        ax1.text(0.02, 0.98, f'Size: {original_size[0]}×{original_size[1]}', 
                 transform=ax1.transAxes, fontsize=9, color='white',
                 bbox=dict(boxstyle='round', facecolor='black', alpha=0.7))
        ax1.axis('off')
        
        ax2 = fig.add_subplot(gs[0, 1])
        if spectral_image is not None:
            spectral_display = cv2.resize(spectral_image, (256, 256))
            spectral_display = self._normalize_for_display(spectral_display)
            original_resized = cv2.resize(original_img_np, (256, 256)) / 255.0
            spectral_importance = float(np.abs(spectral_display - original_resized).mean())
            ax2.imshow(spectral_display)
            ax2.set_title(f'Spectral Component\nImportance: {spectral_importance:.3f}', 
                         fontsize=12, fontweight='bold')
            ax2.text(0.02, 0.98, f'Frequency Features', 
                    transform=ax2.transAxes, fontsize=9, color='white',
                    bbox=dict(boxstyle='round', facecolor='blue', alpha=0.7))
        else:
            ax2.text(0.5, 0.5, 'Spectral\nNot Available', 
                    ha='center', va='center', fontsize=12)
            ax2.set_title('Spectral Component', fontsize=12, fontweight='bold')
        ax2.axis('off')
        
        ax3 = fig.add_subplot(gs[0, 2])
        if vit_attention is not None:
            vit_attn = vit_attention.squeeze().cpu().numpy()
            if vit_attn.ndim == 3:
                vit_attn_2d = vit_attn.mean(axis=0)
                vit_attn_display = cv2.resize(vit_attn_2d, (256, 256))
                vit_attn_display = self._normalize_for_display(vit_attn_display)
                
                im3 = ax3.imshow(vit_attn_display, cmap='hot', alpha=0.8)
                ax3.imshow(cv2.resize(original_img_np, (256, 256)), alpha=0.4)
                
                vit_importance = float(vit_attn_display.mean())
                ax3.set_title(f'ViT Attention\nImportance: {vit_importance:.3f}', 
                             fontsize=12, fontweight='bold')
                ax3.text(0.02, 0.98, f'Global Context Attention', 
                        transform=ax3.transAxes, fontsize=9, color='white',
                        bbox=dict(boxstyle='round', facecolor='red', alpha=0.7))
                
                plt.colorbar(im3, ax=ax3, fraction=0.046, pad=0.04)
        else:
            ax3.text(0.5, 0.5, 'ViT Attention\nNot Available', 
                    ha='center', va='center', fontsize=12)
            ax3.set_title('ViT Attention', fontsize=12, fontweight='bold')
        ax3.axis('off')
        
        ax4 = fig.add_subplot(gs[0, 3])
        if cnn_features is not None:
            cnn_feat = cnn_features.squeeze().cpu().numpy()
            if cnn_feat.ndim == 3:
                cnn_feat_2d = cnn_feat.mean(axis=0)
                cnn_feat_display = cv2.resize(cnn_feat_2d, (256, 256))
                cnn_feat_display = self._normalize_for_display(cnn_feat_display)
                
                im4 = ax4.imshow(cnn_feat_display, cmap='viridis', alpha=0.8)
                ax4.imshow(cv2.resize(original_img_np, (256, 256)), alpha=0.4)
                
                cnn_importance = float(cnn_feat_display.mean())
                ax4.set_title(f'EfficientNet Features\nImportance: {cnn_importance:.3f}', 
                             fontsize=12, fontweight='bold')
                ax4.text(0.02, 0.98, f'Local Texture Features', 
                        transform=ax4.transAxes, fontsize=9, color='white',
                        bbox=dict(boxstyle='round', facecolor='green', alpha=0.7))
                
                plt.colorbar(im4, ax=ax4, fraction=0.046, pad=0.04)
        else:
            ax4.text(0.5, 0.5, 'EfficientNet\nNot Available', 
                    ha='center', va='center', fontsize=12)
            ax4.set_title('EfficientNet Features', fontsize=12, fontweight='bold')
        ax4.axis('off')
        
        ax5 = fig.add_subplot(gs[1, 0])
        if 'final_mask' in all_activations:
            pred_mask = all_activations['final_mask'].squeeze().cpu().numpy()
            if pred_mask.ndim == 2:
                pred_mask_display = cv2.resize(pred_mask, (256, 256))
                ax5.imshow(pred_mask_display, cmap='gray')
                mask_coverage = float((pred_mask_display > 0.5).mean())
                ax5.set_title(f'Predicted Disease Mask\nCoverage: {mask_coverage:.2%}', 
                             fontsize=12, fontweight='bold')
        else:
            ax5.text(0.5, 0.5, 'Predicted Mask\nNot Available', 
                    ha='center', va='center', fontsize=12)
            ax5.set_title('Predicted Disease Mask', fontsize=12, fontweight='bold')
        ax5.axis('off')
        
        ax6 = fig.add_subplot(gs[1, 1:])
        components = ['Spectral', 'ViT Attention', 'EfficientNet']
        importances = [
            spectral_importance if 'spectral_importance' in locals() else 0,
            vit_importance if 'vit_importance' in locals() else 0,
            cnn_importance if 'cnn_importance' in locals() else 0
        ]
        
        colors = ['blue', 'red', 'green']
        bars = ax6.bar(components, importances, color=colors, edgecolor='black', alpha=0.7)
        
        for bar, imp in zip(bars, importances):
            height = bar.get_height()
            ax6.text(bar.get_x() + bar.get_width()/2., height + 0.01,
                    f'{imp:.3f}', ha='center', va='bottom', fontsize=10, fontweight='bold')
        
        ax6.set_ylabel('Importance Score', fontsize=11, fontweight='bold')
        ax6.set_title('Component Importance Comparison', fontsize=13, fontweight='bold')
        ax6.grid(True, alpha=0.3, axis='y')
        ax6.set_ylim([0, max(importances) * 1.2 if importances else 1])
        
        ax7 = fig.add_subplot(gs[2, 0])
        if sum(importances) > 0:
            normalized_imp = [imp/sum(importances) for imp in importances]
            wedges, texts, autotexts = ax7.pie(normalized_imp, labels=components, colors=colors,
                                              autopct='%1.1f%%', startangle=90,
                                              textprops={'fontsize': 10, 'fontweight': 'bold'})
            for autotext in autotexts:
                autotext.set_color('white')
                autotext.set_fontweight('bold')
            ax7.set_title('Relative Importance (%)', fontsize=12, fontweight='bold')
        else:
            ax7.text(0.5, 0.5, 'No importance data\navailable', 
                    ha='center', va='center', fontsize=12)
            ax7.set_title('Relative Importance', fontsize=12, fontweight='bold')
        
        ax8 = fig.add_subplot(gs[2, 1:])
        ax8.axis('off')
        
        info_text = "=" * 60 + "\n"
        info_text += "PREDICTION RESULTS\n"
        info_text += "=" * 60 + "\n\n"
        info_text += f"🌿 Image: {image_name}\n"
        info_text += f"📏 Original Size: {original_size[0]} × {original_size[1]}\n"
        info_text += f"🎯 Predicted Class: {pred_class_name} (Class {pred_class})\n"
        info_text += f"📈 Confidence: {confidence:.2%}\n\n"
        
        if ground_truth_label is not None:
            info_text += f"✅ Ground Truth: {gt_class_name} (Class {ground_truth_label})\n"
            info_text += f"✓ Prediction {'CORRECT' if gt_correct else 'INCORRECT'}\n\n"
        
        info_text += "COMPONENT ANALYSIS\n"
        info_text += "-" * 40 + "\n"
        for comp, imp in zip(components, importances):
            info_text += f"• {comp}: {imp:.3f} "
            if imp > 0.5:
                info_text += "🔥 (High Impact)\n"
            elif imp > 0.2:
                info_text += "⚡ (Moderate Impact)\n"
            else:
                info_text += "💧 (Low Impact)\n"
        
        info_text += "\n" + "=" * 60 + "\n"
        info_text += "💡 Key Insights:\n"
        if 'spectral_importance' in locals() and 'vit_importance' in locals() and 'cnn_importance' in locals():
            if spectral_importance > vit_importance and spectral_importance > cnn_importance:
                info_text += "• Spectral preprocessing is MOST important\n"
                info_text += "• Color/frequency features are crucial for this disease\n"
            elif vit_importance > spectral_importance and vit_importance > cnn_importance:
                info_text += "• ViT attention is MOST important\n"
                info_text += "• Global context and relationships matter most\n"
            else:
                info_text += "• EfficientNet features are MOST important\n"
                info_text += "• Local texture and patterns are key indicators\n"
        
        ax8.text(0.02, 0.98, info_text, transform=ax8.transAxes,
                fontsize=10, fontfamily='monospace', va='top',
                bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.9))
        
        plt.suptitle(f'CAMDF-Net  - Component Importance Analysis: {image_name}', 
                    fontsize=16, fontweight='bold', y=0.98)
        
        plt.tight_layout()
        
        save_dir = self.output_dir / 'component_importance'
        save_dir.mkdir(exist_ok=True)
        save_path = save_dir / f'{image_name}_component_importance.png'
        plt.savefig(str(save_path), dpi=300, bbox_inches='tight', facecolor='white')
        plt.close()
        
        print(f"✅ Component importance visualization saved to: {save_path}")
        
        return {
            'image_name': image_name,
            'save_path': str(save_path),
            'predicted_class': pred_class,
            'predicted_class_name': pred_class_name,
            'confidence': confidence,
            'ground_truth': ground_truth_label,
            'ground_truth_name': gt_class_name,
            'component_importances': dict(zip(components, importances))
        }
    
    def batch_component_analysis(self, dataloader, class_names, num_samples=5):
        print(f"\n📊 Running Batch Component Analysis on {num_samples} samples...")
        
        import pandas as pd
        
        results = []
        all_importances = {'Spectral': [], 'ViT Attention': [], 'EfficientNet': []}
        
        for i, (images, masks, labels, original_sizes) in enumerate(dataloader):
            if i >= num_samples:
                break
            
            images = images.to(self.device)
            label = labels[0].item()
            
            with torch.no_grad():
                outputs = self.model(images[0:1])
                all_activations = self.model.get_all_activations()
            
            probs = F.softmax(outputs['logits'], dim=1).cpu().numpy()[0]
            pred_class = int(np.argmax(probs))
            confidence = float(probs[pred_class])
            
            spectral_image = outputs['spectral_image'].cpu().numpy()[0].transpose(1, 2, 0)
            vit_attention = all_activations.get('vit_spatial', None)
            cnn_features = all_activations.get('cnn_features_raw', None)
            
            spectral_imp = 0.0
            if spectral_image is not None:
                spectral_imp = float(np.abs(spectral_image).mean())
            
            vit_imp = 0.0
            if vit_attention is not None:
                vit_attn = vit_attention.squeeze().cpu().numpy()
                if vit_attn.ndim == 3:
                    vit_imp = float(vit_attn.mean())
            
            cnn_imp = 0.0
            if cnn_features is not None:
                cnn_feat = cnn_features.squeeze().cpu().numpy()
                if cnn_feat.ndim == 3:
                    cnn_imp = float(cnn_feat.mean())
            
            result = {
                'sample_id': i,
                'true_class': class_names[label],
                'pred_class': class_names[pred_class],
                'correct': (label == pred_class),
                'confidence': confidence,
                'spectral_importance': spectral_imp,
                'vit_importance': vit_imp,
                'efficientnet_importance': cnn_imp,
                'dominant_component': max(['Spectral', 'ViT Attention', 'EfficientNet'], 
                                         key=lambda x: [spectral_imp, vit_imp, cnn_imp][['Spectral', 'ViT Attention', 'EfficientNet'].index(x)])
            }
            results.append(result)
            
            all_importances['Spectral'].append(spectral_imp)
            all_importances['ViT Attention'].append(vit_imp)
            all_importances['EfficientNet'].append(cnn_imp)
            
            print(f"  Sample {i+1}: {class_names[label]} → {class_names[pred_class]} "
                  f"(✓{label == pred_class}) | "
                  f"S:{spectral_imp:.3f} V:{vit_imp:.3f} E:{cnn_imp:.3f}")
        
        df = pd.DataFrame(results)
        
        fig, axes = plt.subplots(2, 2, figsize=(14, 10))
        
        ax1 = axes[0, 0]
        avg_importances = {comp: np.mean(vals) for comp, vals in all_importances.items()}
        bars = ax1.bar(avg_importances.keys(), avg_importances.values(), 
                       color=['blue', 'red', 'green'], edgecolor='black', alpha=0.7)
        ax1.set_title('Average Component Importance', fontsize=13, fontweight='bold')
        ax1.set_ylabel('Importance Score', fontsize=11)
        ax1.grid(True, alpha=0.3, axis='y')
        
        for bar, val in zip(bars, avg_importances.values()):
            ax1.text(bar.get_x() + bar.get_width()/2., val + 0.01,
                    f'{val:.3f}', ha='center', va='bottom', fontsize=10, fontweight='bold')
        
        ax2 = axes[0, 1]
        dominant_counts = df['dominant_component'].value_counts()
        ax2.pie(dominant_counts.values, labels=dominant_counts.index,
               colors=['blue', 'red', 'green'], autopct='%1.1f%%',
               startangle=90, textprops={'fontsize': 10, 'fontweight': 'bold'})
        ax2.set_title('Dominant Component Distribution', fontsize=13, fontweight='bold')
        
        ax3 = axes[1, 0]
        correct_df = df[df['correct']]
        incorrect_df = df[~df['correct']]
        
        if not correct_df.empty:
            correct_means = [
                correct_df['spectral_importance'].mean(),
                correct_df['vit_importance'].mean(),
                correct_df['efficientnet_importance'].mean()
            ]
        else:
            correct_means = [0, 0, 0]
        
        if not incorrect_df.empty:
            incorrect_means = [
                incorrect_df['spectral_importance'].mean(),
                incorrect_df['vit_importance'].mean(),
                incorrect_df['efficientnet_importance'].mean()
            ]
        else:
            incorrect_means = [0, 0, 0]
        
        x = np.arange(len(avg_importances))
        width = 0.35
        ax3.bar(x - width/2, correct_means, width, label='Correct Predictions', 
               color=['lightblue', 'lightcoral', 'lightgreen'], edgecolor='black')
        ax3.bar(x + width/2, incorrect_means, width, label='Incorrect Predictions',
               color=['blue', 'red', 'green'], edgecolor='black', alpha=0.6)
        ax3.set_xticks(x)
        ax3.set_xticklabels(list(avg_importances.keys()))
        ax3.set_title('Importance by Prediction Accuracy', fontsize=13, fontweight='bold')
        ax3.set_ylabel('Average Importance')
        ax3.legend()
        ax3.grid(True, alpha=0.3, axis='y')
        
        ax4 = axes[1, 1]
        corr_data = df[['spectral_importance', 'vit_importance', 'efficientnet_importance', 'confidence']]
        corr_matrix = corr_data.corr()
        im = ax4.imshow(corr_matrix, cmap='coolwarm', vmin=-1, vmax=1)
        ax4.set_xticks(range(len(corr_matrix.columns)))
        ax4.set_yticks(range(len(corr_matrix.columns)))
        ax4.set_xticklabels(corr_matrix.columns, rotation=45, ha='right')
        ax4.set_yticklabels(corr_matrix.columns)
        ax4.set_title('Correlation Matrix', fontsize=13, fontweight='bold')
        
        for i in range(len(corr_matrix.columns)):
            for j in range(len(corr_matrix.columns)):
                ax4.text(j, i, f'{corr_matrix.iloc[i, j]:.2f}', 
                        ha='center', va='center', color='white' if abs(corr_matrix.iloc[i, j]) > 0.5 else 'black',
                        fontsize=10, fontweight='bold')
        
        plt.colorbar(im, ax=ax4, fraction=0.046, pad=0.04)
        
        plt.suptitle(f'CAMDF-Net  - Batch Component Analysis ({num_samples} samples)', 
                    fontsize=16, fontweight='bold', y=0.98)
        plt.tight_layout()
        
        save_dir = self.output_dir / 'batch_analysis'
        save_dir.mkdir(exist_ok=True)
        save_path = save_dir / 'component_analysis_summary.png'
        plt.savefig(str(save_path), dpi=300, bbox_inches='tight', facecolor='white')
        plt.close()
        
        csv_path = save_dir / 'component_analysis_stats.csv'
        df.to_csv(csv_path, index=False)
        
        print(f"\n✅ Batch analysis saved to: {save_path}")
        print(f"📊 Statistics saved to: {csv_path}")
        
        print(f"\n📈 Summary Statistics:")
        print(f"  • Average Spectral Importance: {avg_importances['Spectral']:.3f}")
        print(f"  • Average ViT Importance: {avg_importances['ViT Attention']:.3f}")
        print(f"  • Average EfficientNet Importance: {avg_importances['EfficientNet']:.3f}")
        print(f"  • Most dominant component: {dominant_counts.idxmax()} ({dominant_counts.max()}/{num_samples} samples)")
        print(f"  • Accuracy: {df['correct'].mean():.2%}")
        
        return df, avg_importances, save_path
    
    def create_all_summary_visualizations(self, results: Dict[str, Any], class_names: List[str]):
        print("\n📊 Creating summary-level visualizations...")
        summary_dir = self.output_dir / 'summary'
        curves_dir = summary_dir / 'curves'
        matrices_dir = summary_dir / 'matrices'
        for d in [summary_dir, curves_dir, matrices_dir]:
            d.mkdir(parents=True, exist_ok=True)
        
        labels = results['labels']
        preds = results['predictions']
        probs = results['probabilities']
        uncertainties = results['uncertainties']
        
        self._plot_confusion_matrix(labels, preds, class_names, matrices_dir)
        self._plot_normalized_confusion_matrix(labels, preds, class_names, matrices_dir)
        self._plot_roc_curves(labels, probs, class_names, curves_dir)
        self._plot_precision_recall_curves(labels, probs, class_names, curves_dir)
        self._plot_per_class_accuracy(labels, preds, class_names, summary_dir)
        self._plot_per_class_f1(labels, preds, class_names, summary_dir)
        self._plot_uncertainty_distribution(uncertainties, labels, preds, class_names, summary_dir)
        self._plot_confidence_distribution(probs, labels, preds, summary_dir)
        self._plot_radar_chart(results, class_names, summary_dir)
        self._save_classification_report(labels, preds, class_names, summary_dir)
        self._plot_prediction_error_analysis(labels, preds, probs, class_names, summary_dir)
        print("✅ All summary visualizations complete!\n")
    
    def _plot_confusion_matrix(self, labels, preds, class_names, folder):
        cm = confusion_matrix(labels, preds)
        fig, ax = plt.subplots(figsize=(10, 8))
        im = ax.imshow(cm, interpolation='nearest', cmap='Blues')
        ax.set_title('Confusion Matrix', fontsize=16, fontweight='bold', pad=15)
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
        n = len(class_names)
        ax.set_xticks(range(n)); ax.set_yticks(range(n))
        ax.set_xticklabels(class_names, rotation=45, ha='right', fontsize=9)
        ax.set_yticklabels(class_names, fontsize=9)
        ax.set_xlabel('Predicted', fontsize=13, fontweight='bold')
        ax.set_ylabel('True', fontsize=13, fontweight='bold')
        thresh = cm.max() / 2.0
        for i in range(n):
            for j in range(n):
                ax.text(j, i, format(cm[i, j], 'd'), ha='center', va='center',
                       color='white' if cm[i, j] > thresh else 'black', fontsize=11, fontweight='bold')
        plt.tight_layout()
        plt.savefig(str(folder / 'confusion_matrix.png'), dpi=300, bbox_inches='tight', facecolor='white')
        plt.close()
        print("   ✅ Confusion matrix saved")
    
    def _plot_normalized_confusion_matrix(self, labels, preds, class_names, folder):
        cm = confusion_matrix(labels, preds)
        cm_norm = cm.astype('float') / (cm.sum(axis=1, keepdims=True) + 1e-8)
        fig, ax = plt.subplots(figsize=(10, 8))
        im = ax.imshow(cm_norm, interpolation='nearest', cmap='YlOrRd', vmin=0, vmax=1)
        ax.set_title('Normalized Confusion Matrix', fontsize=16, fontweight='bold', pad=15)
        plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
        n = len(class_names)
        ax.set_xticks(range(n)); ax.set_yticks(range(n))
        ax.set_xticklabels(class_names, rotation=45, ha='right', fontsize=9)
        ax.set_yticklabels(class_names, fontsize=9)
        ax.set_xlabel('Predicted', fontsize=13, fontweight='bold')
        ax.set_ylabel('True', fontsize=13, fontweight='bold')
        for i in range(n):
            for j in range(n):
                ax.text(j, i, f'{cm_norm[i, j]:.2f}', ha='center', va='center',
                       color='white' if cm_norm[i, j] > 0.5 else 'black', fontsize=11, fontweight='bold')
        plt.tight_layout()
        plt.savefig(str(folder / 'confusion_matrix_normalized.png'), dpi=300, bbox_inches='tight', facecolor='white')
        plt.close()
        print("   ✅ Normalized confusion matrix saved")
    
    def _plot_roc_curves(self, labels, probs, class_names, folder):
        from sklearn.preprocessing import label_binarize
        n_classes = len(class_names)
        y_bin = label_binarize(labels, classes=range(n_classes))
        if y_bin.shape[1] == 1:
            y_bin = np.hstack([1 - y_bin, y_bin])
        
        fig, ax = plt.subplots(figsize=(10, 8))
        colors = plt.cm.Set1(np.linspace(0, 1, n_classes))
        aucs = []
        for i in range(n_classes):
            fpr, tpr, _ = roc_curve(y_bin[:, i], probs[:, i])
            roc_auc = np.trapz(tpr, fpr)
            aucs.append(roc_auc)
            ax.plot(fpr, tpr, color=colors[i], lw=2, label=f'{class_names[i]} (AUC={roc_auc:.3f})')
        ax.plot([0, 1], [0, 1], 'k--', lw=1.5, alpha=0.5, label='Random')
        ax.set_xlabel('False Positive Rate', fontsize=13, fontweight='bold')
        ax.set_ylabel('True Positive Rate', fontsize=13, fontweight='bold')
        ax.set_title('ROC Curves (One-vs-Rest)', fontsize=16, fontweight='bold', pad=15)
        ax.legend(loc='lower right', fontsize=9, framealpha=0.95)
        ax.set_xlim([0, 1]); ax.set_ylim([0, 1.05])
        ax.grid(True, alpha=0.3, linestyle='--')
        plt.tight_layout()
        plt.savefig(str(folder / 'roc_curves.png'), dpi=300, bbox_inches='tight', facecolor='white')
        plt.close()
        print("   ✅ ROC curves saved")
    
    def _plot_precision_recall_curves(self, labels, probs, class_names, folder):
        from sklearn.preprocessing import label_binarize
        n_classes = len(class_names)
        y_bin = label_binarize(labels, classes=range(n_classes))
        if y_bin.shape[1] == 1:
            y_bin = np.hstack([1 - y_bin, y_bin])
        
        fig, ax = plt.subplots(figsize=(10, 8))
        colors = plt.cm.Set2(np.linspace(0, 1, n_classes))
        for i in range(n_classes):
            prec, rec, _ = precision_recall_curve(y_bin[:, i], probs[:, i])
            ap = average_precision_score(y_bin[:, i], probs[:, i])
            ax.plot(rec, prec, color=colors[i], lw=2, label=f'{class_names[i]} (AP={ap:.3f})')
        ax.set_xlabel('Recall', fontsize=13, fontweight='bold')
        ax.set_ylabel('Precision', fontsize=13, fontweight='bold')
        ax.set_title('Precision-Recall Curves', fontsize=16, fontweight='bold', pad=15)
        ax.legend(loc='lower left', fontsize=9, framealpha=0.95)
        ax.set_xlim([0, 1]); ax.set_ylim([0, 1.05])
        ax.grid(True, alpha=0.3, linestyle='--')
        plt.tight_layout()
        plt.savefig(str(folder / 'precision_recall_curves.png'), dpi=300, bbox_inches='tight', facecolor='white')
        plt.close()
        print("   ✅ Precision-Recall curves saved")
    
    def _plot_per_class_accuracy(self, labels, preds, class_names, folder):
        n = len(class_names)
        per_class_acc = []
        for i in range(n):
            mask = labels == i
            if mask.sum() > 0:
                per_class_acc.append(float((preds[mask] == i).sum()) / float(mask.sum()) * 100)
            else:
                per_class_acc.append(0.0)
        fig, ax = plt.subplots(figsize=(12, 6))
        bars = ax.bar(range(n), per_class_acc, color=plt.cm.Set3(np.linspace(0, 1, n)), edgecolor='black', alpha=0.85)
        ax.set_xticks(range(n)); ax.set_xticklabels(class_names, rotation=45, ha='right', fontsize=10)
        ax.set_ylabel('Accuracy (%)', fontsize=13, fontweight='bold')
        ax.set_title('Per-Class Accuracy', fontsize=16, fontweight='bold', pad=15)
        ax.set_ylim([0, 110]); ax.grid(True, alpha=0.3, axis='y', linestyle='--')
        for b, v in zip(bars, per_class_acc):
            ax.text(b.get_x() + b.get_width()/2., b.get_height() + 1, f'{v:.1f}%',
                   ha='center', va='bottom', fontsize=10, fontweight='bold')
        ax.axhline(y=np.mean(per_class_acc), color='red', linestyle='--', lw=2, label=f'Mean: {np.mean(per_class_acc):.1f}%')
        ax.legend(fontsize=11)
        plt.tight_layout()
        plt.savefig(str(folder / 'per_class_accuracy.png'), dpi=300, bbox_inches='tight', facecolor='white')
        plt.close()
        print("   ✅ Per-class accuracy saved")
    
    def _plot_per_class_f1(self, labels, preds, class_names, folder):
        _, _, f1_per, _ = precision_recall_fscore_support(labels, preds, average=None, zero_division=0)
        n = len(class_names)
        fig, ax = plt.subplots(figsize=(12, 6))
        bars = ax.bar(range(n), f1_per * 100, color=plt.cm.Pastel1(np.linspace(0, 1, n)), edgecolor='black', alpha=0.85)
        ax.set_xticks(range(n)); ax.set_xticklabels(class_names, rotation=45, ha='right', fontsize=10)
        ax.set_ylabel('F1-Score (%)', fontsize=13, fontweight='bold')
        ax.set_title('Per-Class F1-Score', fontsize=16, fontweight='bold', pad=15)
        ax.set_ylim([0, 110]); ax.grid(True, alpha=0.3, axis='y', linestyle='--')
        for b, v in zip(bars, f1_per * 100):
            ax.text(b.get_x() + b.get_width()/2., b.get_height() + 1, f'{v:.1f}%',
                   ha='center', va='bottom', fontsize=10, fontweight='bold')
        plt.tight_layout()
        plt.savefig(str(folder / 'per_class_f1.png'), dpi=300, bbox_inches='tight', facecolor='white')
        plt.close()
        print("   ✅ Per-class F1 saved")
    
    def _plot_uncertainty_distribution(self, uncertainties, labels, preds, class_names, folder):
        unc_flat = uncertainties.flatten()
        correct = (preds == labels)
        fig, axes = plt.subplots(1, 3, figsize=(18, 5))
        axes[0].hist(unc_flat, bins=50, color='steelblue', edgecolor='black', alpha=0.7)
        axes[0].set_title('Uncertainty Distribution', fontsize=12, fontweight='bold')
        axes[0].set_xlabel('Uncertainty'); axes[0].set_ylabel('Count'); axes[0].grid(True, alpha=0.3)
        if correct.sum() > 0:
            axes[1].hist(unc_flat[correct], bins=30, color='green', alpha=0.6, label='Correct', edgecolor='black')
        if (~correct).sum() > 0:
            axes[1].hist(unc_flat[~correct], bins=30, color='red', alpha=0.6, label='Incorrect', edgecolor='black')
        axes[1].set_title('Uncertainty: Correct vs Wrong', fontsize=12, fontweight='bold')
        axes[1].legend(fontsize=10); axes[1].grid(True, alpha=0.3)
        for i, cn in enumerate(class_names):
            mask = labels == i
            if mask.sum() > 0:
                axes[2].hist(unc_flat[mask], bins=20, alpha=0.5, label=cn[:15])
        axes[2].set_title('Uncertainty by Class', fontsize=12, fontweight='bold')
        axes[2].legend(fontsize=8); axes[2].grid(True, alpha=0.3)
        plt.suptitle('Model Uncertainty Analysis', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.savefig(str(folder / 'uncertainty_analysis.png'), dpi=300, bbox_inches='tight', facecolor='white')
        plt.close()
        print("   ✅ Uncertainty analysis saved")
    
    def _plot_confidence_distribution(self, probs, labels, preds, folder):
        max_probs = probs.max(axis=1)
        correct = (preds == labels)
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
        ax1.hist(max_probs, bins=50, color='skyblue', edgecolor='black', alpha=0.7)
        ax1.set_title('Confidence Distribution', fontsize=12, fontweight='bold')
        ax1.set_xlabel('Max Probability'); ax1.set_ylabel('Count'); ax1.grid(True, alpha=0.3)
        ax1.axvline(x=0.5, color='red', linestyle='--', label='50% threshold'); ax1.legend()
        bins_edges = np.linspace(0, 1, 11)
        bin_accs, bin_confs = [], []
        for lo, hi in zip(bins_edges[:-1], bins_edges[1:]):
            mask = (max_probs >= lo) & (max_probs < hi)
            if mask.sum() > 0:
                bin_accs.append(float(correct[mask].mean()))
                bin_confs.append(float(max_probs[mask].mean()))
        if bin_confs:
            ax2.bar(bin_confs, bin_accs, width=0.08, color='coral', edgecolor='black', alpha=0.7, label='Accuracy')
            ax2.plot([0, 1], [0, 1], 'k--', lw=1.5, label='Perfect calibration')
            ax2.set_xlabel('Mean Predicted Confidence'); ax2.set_ylabel('Fraction Correct')
            ax2.set_title('Reliability Diagram', fontsize=12, fontweight='bold')
            ax2.legend(); ax2.grid(True, alpha=0.3)
            ax2.set_xlim([0, 1]); ax2.set_ylim([0, 1.05])
        plt.tight_layout()
        plt.savefig(str(folder / 'confidence_calibration.png'), dpi=300, bbox_inches='tight', facecolor='white')
        plt.close()
        print("   ✅ Confidence calibration saved")
    
    def _plot_radar_chart(self, results, class_names, folder):
        metrics = ['Accuracy', 'Precision', 'Recall', 'F1', 'IoU', 'Dice']
        values = [results['accuracy']*100, results['precision']*100, results['recall']*100,
                  results['f1_score']*100, results['iou']*100, results['dice']*100]
        N = len(metrics)
        angles = np.linspace(0, 2 * np.pi, N, endpoint=False).tolist()
        values_closed = values + [values[0]]
        angles_closed = angles + [angles[0]]
        fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))
        ax.fill(angles_closed, values_closed, color='steelblue', alpha=0.25)
        ax.plot(angles_closed, values_closed, 'o-', color='steelblue', lw=2.5, markersize=8)
        ax.set_xticks(angles); ax.set_xticklabels(metrics, fontsize=11, fontweight='bold')
        ax.set_ylim(0, 105); ax.set_title('Performance Radar', fontsize=16, fontweight='bold', pad=20)
        for angle, val, label in zip(angles, values, metrics):
            ax.text(angle, val + 4, f'{val:.1f}%', ha='center', va='center', fontsize=10, fontweight='bold')
        plt.tight_layout()
        plt.savefig(str(folder / 'performance_radar.png'), dpi=300, bbox_inches='tight', facecolor='white')
        plt.close()
        print("   ✅ Radar chart saved")
    
    def _save_classification_report(self, labels, preds, class_names, folder):
        from sklearn.metrics import classification_report
        report = classification_report(labels, preds, target_names=class_names, zero_division=0)
        with open(str(folder / 'classification_report.txt'), 'w') as f:
            f.write(report)
        fig, ax = plt.subplots(figsize=(12, max(4, len(class_names) * 0.6 + 2)))
        ax.text(0.05, 0.95, report, transform=ax.transAxes, fontsize=11, verticalalignment='top',
               fontfamily='monospace', bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))
        ax.set_title('Classification Report', fontsize=14, fontweight='bold'); ax.axis('off')
        plt.tight_layout()
        plt.savefig(str(folder / 'classification_report.png'), dpi=300, bbox_inches='tight', facecolor='white')
        plt.close()
        print("   ✅ Classification report saved")
    
    def _plot_prediction_error_analysis(self, labels, preds, probs, class_names, folder):
        wrong = labels != preds
        if wrong.sum() == 0:
            print("   ✅ No errors to analyze (perfect accuracy)")
            return
        n = len(class_names)
        error_matrix = np.zeros((n, n))
        for t, p in zip(labels[wrong], preds[wrong]):
            error_matrix[t, p] += 1
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
        im = ax1.imshow(error_matrix, cmap='Reds')
        ax1.set_xticks(range(n)); ax1.set_yticks(range(n))
        ax1.set_xticklabels(class_names, rotation=45, ha='right', fontsize=9)
        ax1.set_yticklabels(class_names, fontsize=9)
        ax1.set_xlabel('Predicted'); ax1.set_ylabel('True')
        ax1.set_title('Error Matrix (misclassifications only)', fontsize=12, fontweight='bold')
        plt.colorbar(im, ax=ax1, fraction=0.046, pad=0.04)
        for i in range(n):
            for j in range(n):
                if error_matrix[i, j] > 0:
                    ax1.text(j, i, f'{int(error_matrix[i, j])}', ha='center', va='center', fontsize=10, fontweight='bold')
        wrong_confs = probs[wrong].max(axis=1)
        ax2.hist(wrong_confs, bins=30, color='salmon', edgecolor='darkred', alpha=0.7)
        ax2.set_title('Confidence of Misclassifications', fontsize=12, fontweight='bold')
        ax2.set_xlabel('Confidence'); ax2.set_ylabel('Count'); ax2.grid(True, alpha=0.3)
        ax2.axvline(x=np.mean(wrong_confs), color='black', linestyle='--', label=f'Mean: {np.mean(wrong_confs):.3f}')
        ax2.legend()
        plt.suptitle('Prediction Error Analysis', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.savefig(str(folder / 'error_analysis.png'), dpi=300, bbox_inches='tight', facecolor='white')
        plt.close()
        print("   ✅ Error analysis saved")
    
    def _visualize_vit_headwise(self, image_tensor, folders, image_name, original_size):
        print(f"      🧠 Visualizing ViT head-wise attention...")
        tw = (int(original_size[0]), int(original_size[1]))
        head_attns = self.model.get_vit_self_attention(image_tensor, return_all_heads=True)
        for layer_name, heads in head_attns.items():
            if isinstance(heads, dict):
                for head_name, attn_map in heads.items():
                    head_idx = int(head_name.split('_')[-1]) if head_name.split('_')[-1].isdigit() else 0
                    hf = folders['4b_vit_headwise'] / f'head_{head_idx:02d}'
                    hf.mkdir(exist_ok=True)
                    am = attn_map.squeeze().cpu().float().numpy()
                    if am.ndim == 2:
                        ar = self._safe_resize_2d(am, tw)
                        plt.imsave(str(hf / f'{layer_name}_{head_name}.png'), self._normalize_for_display(ar), cmap='magma')
                fig, axes = plt.subplots(3, 4, figsize=(16, 12))
                axes = axes.flatten()
                for idx, (hn, am) in enumerate(heads.items()):
                    if idx >= 12: break
                    a = am.squeeze().cpu().float().numpy()
                    if a.ndim == 2:
                        axes[idx].imshow(self._normalize_for_display(self._safe_resize_2d(a, tw)), cmap='magma')
                    axes[idx].set_title(hn, fontsize=8, fontweight='bold'); axes[idx].axis('off')
                for idx in range(len(heads), 12): axes[idx].axis('off')
                plt.suptitle(f'ViT Self-Attention Heads: {layer_name}', fontsize=12, fontweight='bold')
                plt.tight_layout()
                plt.savefig(str(folders['4b_vit_headwise'] / f'{image_name}_{layer_name}_all_heads.png'),
                           dpi=300, bbox_inches='tight', facecolor='white')
                plt.close()
    
    def _visualize_segmentation_layers(self, all_activations, folders, image_name, original_size):
        print(f"      🎭 Visualizing segmentation layers...")
        tw = (int(original_size[0]), int(original_size[1]))
        seg_keys = sorted([k for k in all_activations.keys() if k.startswith('seg_layer_')])
        for aname in seg_keys:
            t = all_activations[aname]
            if t is None: continue
            layer_num = aname.split('_')[-1]
            lf = folders['6d_segmentation_layers'] / f'layer_{int(layer_num):03d}'
            lf.mkdir(exist_ok=True)
            self.visualize_activation_layer(t, aname, lf, original_size, 'summer')
        mask_keys = [k for k in all_activations.keys() if any(w in k.lower() for w in ['mask_logits', 'refined_mask', 'final_mask', 'gated_x'])]
        if mask_keys:
            cols = min(4, len(mask_keys))
            rows = (len(mask_keys) + cols - 1) // cols
            fig, axes = plt.subplots(rows, cols, figsize=(cols * 5, rows * 4))
            if rows == 1 and cols == 1: axes = np.array([[axes]])
            elif rows == 1: axes = axes.reshape(1, -1)
            elif cols == 1: axes = axes.reshape(-1, 1)
            for idx, mn in enumerate(mask_keys):
                r, c = idx // cols, idx % cols
                mt = t.squeeze().cpu().float().numpy()
                if mt.ndim == 3: mt = mt.mean(axis=0)
                elif mt.ndim < 2: axes[r, c].text(0.5, 0.5, f'{mn}', ha='center', va='center'); axes[r, c].axis('off'); continue
                mr = self._safe_resize_2d(mt, tw)
                axes[r, c].imshow(self._normalize_for_display(mr), cmap='bone')
                axes[r, c].set_title(mn[:25], fontsize=9, fontweight='bold'); axes[r, c].axis('off')
            for idx in range(len(mask_keys), rows * cols): axes[idx // cols, idx % cols].axis('off')
            plt.suptitle(f'Mask Generation Progression: {image_name}', fontsize=12, fontweight='bold')
            plt.tight_layout()
            plt.savefig(str(folders['6c_mask_generation'] / f'{image_name}_mask_progression.png'),
                       dpi=300, bbox_inches='tight', facecolor='white')
            plt.close()
    
    def _visualize_classifier_features(self, all_activations, folders, image_name):
        print(f"      📊 Visualizing classifier features...")
        cls_keys = [k for k in all_activations.keys() if any(w in k.lower() for w in ['classifier', 'cls_logits', 'uncertainty'])]
        for aname in cls_keys:
            t = all_activations[aname]
            if t is None: continue
            anp = t.squeeze().cpu().float().numpy()
            if anp.ndim >= 1:
                fig, ax = plt.subplots(figsize=(10, 5))
                if anp.ndim == 1:
                    bars = ax.bar(range(len(anp)), anp, color='coral', edgecolor='black', alpha=0.7)
                    for b, v in zip(bars, anp):
                        ax.text(b.get_x() + b.get_width()/2., b.get_height() + 0.01, f'{v:.3f}',
                               ha='center', fontsize=9)
                else:
                    ax.imshow(self._normalize_for_display(anp), cmap='spring', aspect='auto')
                ax.set_title(f'{aname}', fontsize=11, fontweight='bold'); ax.grid(True, alpha=0.3)
                plt.tight_layout()
                plt.savefig(str(folders['7a_classifier_features'] / f'{image_name}_{aname}.png'),
                           dpi=300, bbox_inches='tight', facecolor='white')
                plt.close()
    
    def _create_feature_comparison_grid(self, all_activations, folders, image_name, original_size):
        print(f"      📐 Creating feature comparison grid...")
        tw = (int(original_size[0]), int(original_size[1]))
        feat_keys = [k for k in all_activations.keys() if any(w in k.lower() for w in ['cnn_features', 'vit_spatial', 'fused_features', 'cnn_aligned'])]
        feat_keys = [k for k in feat_keys if all_activations[k] is not None]
        if not feat_keys: return
        fig, axes = plt.subplots(1, len(feat_keys), figsize=(5 * len(feat_keys), 5))
        if len(feat_keys) == 1: axes = [axes]
        cmaps = ['plasma', 'magma', 'coolwarm', 'viridis']
        for idx, fn in enumerate(feat_keys):
            t = all_activations[fn].squeeze().cpu().float().numpy()
            if t.ndim == 3: t = t.mean(axis=0)
            elif t.ndim != 2: axes[idx].text(0.5, 0.5, fn[:20], ha='center', va='center'); axes[idx].axis('off'); continue
            axes[idx].imshow(self._normalize_for_display(self._safe_resize_2d(t, tw)), cmap=cmaps[idx % len(cmaps)])
            axes[idx].set_title(fn[:25], fontsize=10, fontweight='bold'); axes[idx].axis('off')
        plt.suptitle(f'Feature Comparison: {image_name}', fontsize=13, fontweight='bold')
        plt.tight_layout()
        plt.savefig(str(folders['10b_feature_grids'] / f'{image_name}_feature_comparison.png'),
                   dpi=300, bbox_inches='tight', facecolor='white')
        plt.close()
    
    def _create_attention_grid(self, all_activations, folders, image_name, original_size):
        print(f"      🔍 Creating attention comparison grid...")
        tw = (int(original_size[0]), int(original_size[1]))
        attn_keys = [k for k in all_activations.keys() if 'attention' in k.lower() and all_activations[k] is not None]
        if not attn_keys: return
        n = min(6, len(attn_keys))
        cols = min(3, n); rows = (n + cols - 1) // cols
        fig, axes = plt.subplots(rows, cols, figsize=(5 * cols, 4 * rows))
        if rows == 1 and cols == 1: axes = np.array([[axes]])
        elif rows == 1: axes = axes.reshape(1, -1)
        elif cols == 1: axes = axes.reshape(-1, 1)
        for idx, an in enumerate(attn_keys[:n]):
            r, c = idx // cols, idx % cols
            t = all_activations[an].squeeze().cpu().float().numpy()
            if t.ndim == 3: t = t.mean(axis=0)
            elif t.ndim != 2: axes[r, c].text(0.5, 0.5, an[:20], ha='center', va='center'); axes[r, c].axis('off'); continue
            axes[r, c].imshow(self._normalize_for_display(self._safe_resize_2d(t, tw)), cmap='hot')
            axes[r, c].set_title(an[:25], fontsize=9, fontweight='bold'); axes[r, c].axis('off')
        for idx in range(n, rows * cols): axes[idx // cols, idx % cols].axis('off')
        plt.suptitle(f'Attention Comparison: {image_name}', fontsize=13, fontweight='bold')
        plt.tight_layout()
        plt.savefig(str(folders['10c_attention_grids'] / f'{image_name}_attention_grid.png'),
                   dpi=300, bbox_inches='tight', facecolor='white')
        plt.close()
    
    def _save_gradient_stats(self, gradients, folders, image_name):
        if not gradients: return
        print(f"      📉 Saving gradient statistics...")
        stats = {}
        for n, g in gradients.items():
            if g is not None:
                gnp = g.cpu().float().numpy()
                stats[n] = {'mean': float(np.mean(gnp)), 'std': float(np.std(gnp)),
                           'min': float(np.min(gnp)), 'max': float(np.max(gnp)),
                           'l2_norm': float(np.sqrt(np.sum(gnp**2))), 'shape': list(gnp.shape)}
        with open(str(folders['11b_gradient_stats'] / f'{image_name}_gradient_stats.json'), 'w') as f:
            json.dump(stats, f, indent=2)
        src = folders['3c_cnn_gradients'] / f'{image_name}_gradient_analysis.png'
        dst = folders['12b_gradient_flow'] / f'{image_name}_gradient_flow.png'
        if src.exists():
            import shutil
            shutil.copy2(str(src), str(dst))
    
    def _save_per_image_metrics(self, image_name, class_name, confidence, severity, folders):
        metrics = {'image': image_name, 'class': class_name, 'confidence': confidence, 'severity': severity}
        with open(str(folders['11c_performance_metrics'] / f'{image_name}_metrics.json'), 'w') as f:
            json.dump(metrics, f, indent=2)
        fig, ax = plt.subplots(figsize=(8, 4))
        text = (f"Image: {image_name}\nPredicted: {class_name}\nConfidence: {confidence:.2%}\n"
                f"Severity: {severity['severity_level']}\nAffected: {severity['affected_percentage']:.1f}%\n"
                f"Lesions: {severity['num_lesions']}")
        ax.text(0.1, 0.5, text, fontsize=13, va='center', fontfamily='monospace',
               bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.9))
        ax.set_title('Per-Image Metrics', fontsize=14, fontweight='bold'); ax.axis('off')
        plt.tight_layout()
        plt.savefig(str(folders['11c_performance_metrics'] / f'{image_name}_metrics.png'),
                   dpi=300, bbox_inches='tight', facecolor='white')
        plt.close()
    
    def _save_severity_visualization(self, pred_mask_uint8, severity, folders, image_name):
        print(f"      ⚕️  Saving severity visualization...")
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
        ax1.imshow(pred_mask_uint8, cmap='hot')
        ax1.set_title(f'Disease Map: {severity["severity_level"]}', fontsize=12, fontweight='bold'); ax1.axis('off')
        levels = ['Healthy', 'Minimal', 'Mild', 'Moderate', 'Severe', 'Critical']
        colors_sev = ['#2ecc71', '#27ae60', '#f1c40f', '#e67e22', '#e74c3c', '#c0392b']
        current_idx = levels.index(severity['severity_level']) if severity['severity_level'] in levels else 0
        bar_colors = ['lightgray'] * len(levels)
        bar_colors[current_idx] = colors_sev[current_idx]
        ax2.barh(range(len(levels)), [100/len(levels)] * len(levels), color=bar_colors, edgecolor='black', alpha=0.7)
        ax2.set_yticks(range(len(levels))); ax2.set_yticklabels(levels, fontsize=11)
        ax2.set_title(f'Severity: {severity["affected_percentage"]:.1f}% affected', fontsize=12, fontweight='bold')
        ax2.axhline(y=current_idx, color=colors_sev[current_idx], lw=3, linestyle='--')
        ax2.grid(True, alpha=0.3, axis='x')
        plt.suptitle(f'Disease Severity: {image_name}', fontsize=13, fontweight='bold')
        plt.tight_layout()
        plt.savefig(str(folders['11d_disease_severity'] / f'{image_name}_severity.png'),
                   dpi=300, bbox_inches='tight', facecolor='white')
        plt.close()
        with open(str(folders['11d_disease_severity'] / f'{image_name}_severity.json'), 'w') as f:
            json.dump(severity, f, indent=2)
    
    def _save_model_state_snapshot(self, folders, image_name):
        summary = {
            'model_name': self.config.model.name,
            'num_parameters': sum(p.numel() for p in self.model.parameters()),
            'num_trainable': sum(p.numel() for p in self.model.parameters() if p.requires_grad),
            'hidden_dim': self.model.hidden_dim,
            'num_heads': self.model.num_heads,
            'num_activations_stored': len(self.model.activations),
            'activation_keys': list(self.model.activations.keys()),
        }
        with open(str(folders['12c_model_states'] / f'{image_name}_model_state.json'), 'w') as f:
            json.dump(summary, f, indent=2)
    
    def _save_normalized_input(self, image_tensor, folders, image_name, original_size):
        inp = image_tensor.squeeze(0).cpu().numpy().transpose(1, 2, 0)
        inp_norm = self._normalize_for_display(inp)
        tw = (int(original_size[0]), int(original_size[1]))
        inp_resized = cv2.resize(inp_norm, tw, interpolation=cv2.INTER_CUBIC)
        plt.imsave(str(folders['2b_normalized'] / f'{image_name}_normalized.png'), np.clip(inp_resized, 0, 1))
        ch_names = ['Red (norm)', 'Green (norm)', 'Blue (norm)']
        fig, axes = plt.subplots(1, 3, figsize=(15, 4))
        for i in range(3):
            ch = inp[:, :, i]
            ch_r = cv2.resize(ch, tw, interpolation=cv2.INTER_CUBIC)
            axes[i].imshow(ch_r, cmap='gray'); axes[i].set_title(ch_names[i], fontsize=11, fontweight='bold'); axes[i].axis('off')
        plt.suptitle(f'Normalized Input Channels: {image_name}', fontsize=12, fontweight='bold')
        plt.tight_layout()
        plt.savefig(str(folders['2b_normalized'] / f'{image_name}_channels.png'), dpi=300, bbox_inches='tight', facecolor='white')
        plt.close()
    
    def _save_fused_feature_viz(self, all_activations, folders, image_name, original_size):
        tw = (int(original_size[0]), int(original_size[1]))
        fused = all_activations.get('fused_features')
        if fused is None: return
        fnp = fused.squeeze().cpu().float().numpy()
        if fnp.ndim == 3:
            nc = min(fnp.shape[0], 16)
            cols, rows = 4, (nc + 3) // 4
            fig, axes = plt.subplots(rows, cols, figsize=(cols * 4, rows * 3))
            if rows == 1 and cols == 1: axes = np.array([[axes]])
            elif rows == 1: axes = axes.reshape(1, -1)
            elif cols == 1: axes = axes.reshape(-1, 1)
            for idx in range(nc):
                r, c = idx // cols, idx % cols
                ch = self._normalize_for_display(self._safe_resize_2d(fnp[idx], tw))
                axes[r, c].imshow(ch, cmap='coolwarm'); axes[r, c].set_title(f'Fused Ch {idx}', fontsize=8, fontweight='bold'); axes[r, c].axis('off')
            for idx in range(nc, rows * cols): axes[idx // cols, idx % cols].axis('off')
            plt.suptitle(f'Fused Features: {image_name}', fontsize=12, fontweight='bold')
            plt.tight_layout()
            plt.savefig(str(folders['5b_fused_features'] / f'{image_name}_fused_grid.png'), dpi=300, bbox_inches='tight', facecolor='white')
            plt.close()
            mean_fused = fnp.mean(axis=0)
            plt.imsave(str(folders['5b_fused_features'] / f'{image_name}_fused_mean.png'),
                       self._normalize_for_display(self._safe_resize_2d(mean_fused, tw)), cmap='coolwarm')
    
    def _save_cnn_vit_statistics(self, all_activations, folders, image_name):
        for prefix, folder_key in [('cnn', '3d_cnn_statistics'), ('vit', '4e_vit_statistics')]:
            keys = [k for k in all_activations.keys() if prefix in k.lower() and all_activations[k] is not None]
            if not keys: continue
            stats_list = []
            for k in keys:
                t = all_activations[k].cpu().float()
                stats_list.append({'name': k, 'shape': list(t.shape), 'mean': float(t.mean()),
                                   'std': float(t.std()), 'min': float(t.min()), 'max': float(t.max()),
                                   'numel': int(t.numel())})
            with open(str(folders[folder_key] / f'{image_name}_{prefix}_stats.json'), 'w') as f:
                json.dump(stats_list, f, indent=2)
            fig, ax = plt.subplots(figsize=(12, 5))
            names = [s['name'][:20] for s in stats_list]
            means = [s['mean'] for s in stats_list]
            stds = [s['std'] for s in stats_list]
            ax.bar(range(len(names)), means, yerr=stds, color='steelblue' if prefix == 'cnn' else 'mediumpurple',
                   edgecolor='black', alpha=0.7, capsize=3)
            ax.set_xticks(range(len(names))); ax.set_xticklabels(names, rotation=45, ha='right', fontsize=8)
            ax.set_title(f'{prefix.upper()} Activation Statistics: {image_name}', fontsize=12, fontweight='bold')
            ax.set_ylabel('Mean ± Std'); ax.grid(True, alpha=0.3, axis='y')
            plt.tight_layout()
            plt.savefig(str(folders[folder_key] / f'{image_name}_{prefix}_stats.png'), dpi=300, bbox_inches='tight', facecolor='white')
            plt.close()
    
    def _save_metadata_extras(self, folders, image_name, config, class_name, confidence, severity, saved_paths):
        from dataclasses import asdict
        with open(str(folders['13a_configurations'] / f'{image_name}_config.json'), 'w') as f:
            json.dump(asdict(config), f, indent=2)
        pred_data = {'image': image_name, 'class': class_name, 'confidence': confidence, 'severity': severity}
        with open(str(folders['13b_predictions'] / f'{image_name}_prediction.json'), 'w') as f:
            json.dump(pred_data, f, indent=2)
        report = (f"CAMDF-Net  Analysis Report\n{'='*40}\n"
                  f"Image: {image_name}\nPredicted: {class_name} ({confidence:.2%})\n"
                  f"Severity: {severity['severity_level']} ({severity['affected_percentage']:.1f}%)\n"
                  f"Lesions: {severity['num_lesions']}\n"
                  f"Recommendation: {self.generate_recommendation(severity, confidence)}\n"
                  f"Generated: {datetime.datetime.now().isoformat()}\n")
        with open(str(folders['13c_analysis_reports'] / f'{image_name}_report.txt'), 'w') as f:
            f.write(report)
        log = {'image': image_name, 'timestamp': datetime.datetime.now().isoformat(),
               'files_generated': list(saved_paths.keys()),
               'total_visualizations': self.count_visualizations(folders['root'])}
        with open(str(folders['13d_visualization_logs'] / f'{image_name}_viz_log.json'), 'w') as f:
            json.dump(log, f, indent=2)

    def visualize_all_activations_and_features(self, image_path, class_names,
                                              ground_truth_mask=None,
                                              ground_truth_original_size=None):
        print(f"\n{'='*80}")
        print(f"🌟 VISUALIZATION FOR: {Path(image_path).name}")
        print(f"{'='*80}")
        
        original_image = Image.open(image_path).convert('RGB')
        original_size = original_image.size
        original_img_np = np.array(original_image)
        image_name = Path(image_path).stem
        
        print(f"    📁 Creating folder structure...")
        folders = self.create_comprehensive_image_folder_structure(image_name)
        saved_paths = {'image_name': image_name}
        
        plt.imsave(str(folders['0_original'] / f'{image_name}_original.png'), original_img_np)
        saved_paths['original'] = str(folders['0_original'] / f'{image_name}_original.png')
        
        if ground_truth_mask is not None:
            gmu = (ground_truth_mask * 255).astype(np.uint8) if ground_truth_mask.max() <= 1 else ground_truth_mask.astype(np.uint8)
            plt.imsave(str(folders['8b_ground_truth'] / f'{image_name}_gt.png'), gmu, cmap='gray')
            saved_paths['ground_truth'] = str(folders['8b_ground_truth'] / f'{image_name}_gt.png')
        
        transform = transforms.Compose([
            transforms.Resize((self.config.dataset.image_size, self.config.dataset.image_size)),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        image_tensor = transform(original_image).unsqueeze(0).to(self.device)
        
        print(f"    🔍 Forward pass...")
        with torch.no_grad():
            outputs = self.model(image_tensor)
            all_activations = self.model.get_all_activations()
        
        inp = image_tensor.squeeze(0).cpu().numpy().transpose(1, 2, 0)
        inp = np.clip(inp * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406]), 0, 1)
        inp_resized = np.clip(cv2.resize(inp, original_size, interpolation=cv2.INTER_CUBIC), 0, 1)
        plt.imsave(str(folders['1_inputs'] / f'{image_name}_input.png'), inp_resized)
        saved_paths['model_input'] = str(folders['1_inputs'] / f'{image_name}_input.png')
        
        probs = F.softmax(outputs['logits'], dim=1).cpu().numpy()[0]
        pred_class = int(np.argmax(probs))
        confidence = float(probs[pred_class])
        class_name = class_names[pred_class]
        
        pred_mask = torch.sigmoid(outputs['mask']).cpu().numpy()[0, 0]
        pred_mask_resized = cv2.resize(pred_mask, original_size, interpolation=cv2.INTER_LINEAR)
        pred_mask_uint8 = (pred_mask_resized * 255).astype(np.uint8)
        
        plt.imsave(str(folders['8a_predicted_masks'] / f'{image_name}_pred_mask.png'), pred_mask_uint8, cmap='gray')
        saved_paths['predicted_mask'] = str(folders['8a_predicted_masks'] / f'{image_name}_pred_mask.png')
        
        mc = cv2.applyColorMap(pred_mask_uint8, cv2.COLORMAP_JET)
        overlay = cv2.addWeighted(original_img_np, 0.6, mc, 0.4, 0)
        plt.imsave(str(folders['8c_overlays'] / f'{image_name}_overlay.png'), overlay)
        saved_paths['overlay'] = str(folders['8c_overlays'] / f'{image_name}_overlay.png')
        
        print(f"    🎨 Generating ALL activation visualizations...")
        self.create_comprehensive_activation_report(all_activations, folders, image_name, original_size)
        
        all_gradients = self.model.get_all_gradients()
        if all_gradients:
            print(f"    📉 Gradient flow...")
            self.visualize_gradient_flow(all_gradients, folders, image_name)
            self._save_gradient_stats(all_gradients, folders, image_name)
        
        print(f"    ⚕️  Severity analysis...")
        severity = self.compute_disease_severity(pred_mask_uint8)
        
        print(f"    🧠 Populating all per-image visualization folders...")
        self._visualize_vit_headwise(image_tensor, folders, image_name, original_size)
        self._visualize_segmentation_layers(all_activations, folders, image_name, original_size)
        self._visualize_classifier_features(all_activations, folders, image_name)
        self._create_feature_comparison_grid(all_activations, folders, image_name, original_size)
        self._create_attention_grid(all_activations, folders, image_name, original_size)
        self._save_per_image_metrics(image_name, class_name, confidence, severity, folders)
        self._save_severity_visualization(pred_mask_uint8, severity, folders, image_name)
        self._save_model_state_snapshot(folders, image_name)
        self._save_normalized_input(image_tensor, folders, image_name, original_size)
        self._save_fused_feature_viz(all_activations, folders, image_name, original_size)
        self._save_cnn_vit_statistics(all_activations, folders, image_name)
        
        metadata = {
            'image_name': image_name, 'original_size': {'w': original_size[0], 'h': original_size[1]},
            'predicted_class': pred_class, 'class_name': class_name, 'confidence': confidence,
            'probabilities': probs.tolist(),
            'uncertainty': float(outputs['uncertainty'].cpu().numpy()[0, 0]),
            'severity': severity, 'num_activations': len(all_activations),
            'activation_names': list(all_activations.keys()),
            'timestamp': datetime.datetime.now().isoformat(), 'version': 'CAMDF-Net '
        }
        with open(str(folders['13_metadata'] / f'{image_name}_metadata.json'), 'w') as f:
            json.dump(metadata, f, indent=2)
        saved_paths['metadata'] = str(folders['13_metadata'] / f'{image_name}_metadata.json')
        
        print(f"    🎨 Creating ULTIMATE grid...")
        ug = self.create_ultimate_comprehensive_grid(
            image_name, original_img_np, pred_mask_uint8, ground_truth_mask,
            all_activations, pred_class, class_name, confidence, severity,
            folders, original_size)
        saved_paths['ultimate_grid'] = ug
        
        self._save_metadata_extras(folders, image_name, self.config, class_name, confidence, severity, saved_paths)
        
        self.generate_visualization_summary(folders, image_name, saved_paths)
        total_viz = self.count_visualizations(folders['root'])
        
        print(f"\n{'='*80}")
        print(f"✅ COMPLETE! Folder: {folders['root']}")
        print(f"📊 Visualizations: {total_viz} | 🎯 {class_name} ({confidence:.2%})")
        print(f"⚕️  {severity['severity_level']} ({severity['affected_percentage']:.1f}%) | 📈 {len(all_activations)} activations")
        print(f"{'='*80}\n")
        return saved_paths

# ============================================================================
# MAIN EXECUTION
# ============================================================================
def main(config_path: Optional[str] = None):
    print("\n" + "="*80)
    print("CAMDF-NET   - ULTIMATE VISUALIZATION PIPELINE".center(80))
    print("No Resizing in Evaluation | All Activations Saved | Beautiful Visuals".center(80))
    print("Component Importance Analysis | Spectral + ViT + EfficientNet".center(80))
    print("="*80 + "\n")
    
    if config_path and Path(config_path).exists():
        config = CAMDFNetConfig.from_yaml(config_path)
        print(f"✅    Loaded configuration from: {config_path}")
    else:
        config = CAMDFNetConfig()
        print("✅    Using default configuration")
    
    config.dataset.preserve_original_size_eval = True
    config.system.save_all_intermediates = True
    config.model.save_all_activations = True
    
    torch.manual_seed(config.system.seed)
    np.random.seed(config.system.seed)
    
    config_save_path = Path(config.system.log_dir) / 'config_used.yaml'
    config_save_path.parent.mkdir(parents=True, exist_ok=True)
    config.to_yaml(str(config_save_path))
    print(f"✅    Configuration saved to: {config_save_path}\n")
    
    print("="*80)
    print("PHASE 1: TRAINING".center(80))
    print("="*80 + "\n")
    
    trainer = CAMDFNetTrainer(config)
    trainer.train()
    
    print("\n" + "="*80)
    print("PHASE 2: COMPREHENSIVE EVALUATION".center(80))
    print("="*80 + "\n")
    
    best_ckpt = Path(config.system.checkpoint_dir) / 'best.pth'
    if best_ckpt.exists():
        print(f"🤖     Loading best model from: {best_ckpt}")
        ckpt = torch.load(best_ckpt, map_location=trainer.device, weights_only=False)
        trainer.model.load_state_dict(ckpt['model_state_dict'])
        print("✅    Best model loaded\n")
    else:
        print("⚠️    Best checkpoint not found, using current model\n")
    
    evaluator = CAMDFNetEvaluator(trainer.model, config)
    class_names = trainer.train_loader.dataset.dataset.classes

    print("Running comprehensive evaluation...")
    results = evaluator.evaluate(trainer.test_loader)
    print(f"\n{'='*80}")
    print("EVALUATION RESULTS".center(80))
    print(f"{'='*80}")
    print(f"  Accuracy:  {results['accuracy']:.4f} ({results['accuracy']*100:.2f}%)")
    print(f"  Precision: {results['precision']:.4f}")
    print(f"  Recall:    {results['recall']:.4f}")
    print(f"  F1-Score:  {results['f1_score']:.4f}")
    print(f"  IoU:       {results['iou']:.4f}")
    print(f"  Dice:      {results['dice']:.4f}")
    print(f"{'='*80}\n")
    
    print("\n📊 Generating summary-level visualizations...")
    evaluator.create_all_summary_visualizations(results, class_names)
    
    print("\n" + "="*80)
    print("PHASE 3: PER-IMAGE ANALYSIS".center(80))
    print("="*80 + "\n")
    
    test_dataset = trainer.test_loader.dataset
    base_dataset = test_dataset.dataset
    subset_indices = test_dataset.indices
    
    sample_count = 0
    processed_classes = set()
    
    print(f"🚀 Processing sample images...\n")
    
    for subset_idx in range(len(test_dataset)):
        if sample_count >= 16:
            break
            
        img_tensor, gt_mask_tensor, label, original_size = test_dataset[subset_idx]
        
        if label in processed_classes:
            continue

        base_idx = subset_indices[subset_idx]
        img_path = base_dataset.samples[base_idx][0]
        
        print(f"📸    Image {sample_count + 1}: {Path(img_path).name}")
        print(f"      Class: {class_names[label]}, Size: {original_size}")
        
        gt_mask_np = gt_mask_tensor.squeeze(0).numpy()
        
        saved_paths = evaluator.visualize_all_activations_and_features(
            img_path, class_names,
            ground_truth_mask=gt_mask_np,
            ground_truth_original_size=original_size
        )
            
        print(f"      ✅    Saved to: per_image_results/{saved_paths['image_name']}/\n")
            
        processed_classes.add(label)
        sample_count += 1
    
    print("\n" + "="*80)
    print("PHASE 4: COMPONENT IMPORTANCE ANALYSIS".center(80))
    print("="*80 + "\n")
    
    print("🔬 Analyzing component importance for sample images...\n")
    
    sample_count = 0
    component_results = []
    
    for subset_idx in range(len(test_dataset)):
        if sample_count >= 4:
            break
        
        img_tensor, gt_mask_tensor, label, original_size = test_dataset[subset_idx]
        base_idx = subset_indices[subset_idx]
        img_path = base_dataset.samples[base_idx][0]
        
        print(f"📊 Analyzing: {Path(img_path).name}")
        print(f"  Ground Truth: {class_names[label]}")
        
        result = evaluator.visualize_component_importance(
            img_path, 
            class_names, 
            ground_truth_label=label
        )
        
        print(f"  Predicted: {result['predicted_class_name']} ({result['confidence']:.2%})")
        print(f"  Component Importances:")
        for comp, imp in result['component_importances'].items():
            print(f"    • {comp}: {imp:.3f}")
        print()
        
        component_results.append(result)
        sample_count += 1
    
    print("📈 Running batch component analysis...")
    import pandas as pd
    df, avg_importances, analysis_path = evaluator.batch_component_analysis(
        trainer.test_loader, 
        class_names, 
        num_samples=max(10, len(trainer.test_loader))
    )
    
    print("\n" + "="*80)
    print("PIPELINE COMPLETE - CAMDF-NET ".center(80))
    print("="*80 + "\n")
    
    print("📦     Output Structure:")
    print(f"  📊  Summary:           {config.system.output_dir}/summary/")
    print(f"  📁  Per-Image:        {config.system.output_dir}/per_image_results/")
    print(f"  🔬  Component Analysis:{config.system.output_dir}/component_importance/")
    print(f"  📈  Batch Analysis:   {config.system.output_dir}/batch_analysis/")
    print(f"  💾  Checkpoints:      {config.system.checkpoint_dir}")
    print(f"  📈  Training Logs:    {config.system.log_dir}")
    
    print(f"\n🔬  Component Importance Summary:")
    print(f"  • Spectral Component:     {avg_importances['Spectral']:.3f}")
    print(f"  • ViT Attention:          {avg_importances['ViT Attention']:.3f}")
    print(f"  • EfficientNet Features:  {avg_importances['EfficientNet']:.3f}")
    
    if 'Spectral' in avg_importances and 'ViT Attention' in avg_importances and 'EfficientNet' in avg_importances:
        most_important = max(avg_importances, key=avg_importances.get)
        print(f"  • Most Important:        {most_important}")
        
        print(f"\n💡  Key Insights:")
        if most_important == 'Spectral':
            print(f"  • Spectral preprocessing is CRUCIAL for cotton disease detection")
            print(f"  • Color and frequency features are key indicators")
            print(f"  • Consider enhancing spectral analysis further")
        elif most_important == 'ViT Attention':
            print(f"  • Global context (ViT attention) is most important")
            print(f"  • Disease patterns and spatial relationships matter")
            print(f"  • Consider increasing ViT capacity")
        else:
            print(f"  • Local texture features (EfficientNet) are most important")
            print(f"  • Disease spots and local patterns are key")
            print(f"  • Consider deeper CNN backbone")
    
    print(f"\n📊  Performance Summary:")
    print(f"  • Accuracy:  {results['accuracy']*100:.2f}%")
    print(f"  • F1-Score:  {results['f1_score']*100:.2f}%")
    print(f"  • IoU:       {results['iou']*100:.2f}%")
    print(f"  • Dice:      {results['dice']*100:.2f}%")
    
    print("\n" + "="*80)
    print("🎉 ALL DONE! Results ready for analysis.".center(80))
    print("="*80 + "\n")

if __name__ == '__main__':
    main()